# DISCLAIMER



---

Esse documento ainda precisa passar por revisões e ser aprovado para que possa ser publicado e compartilhado.

Esse documento ainda não representa a ideia final e está sujeito a mudanças.

---



# ONDE ESTÁVAMOS?

Essa é a continuação do material. Caso esteja perdido clique aqui: [link text](https://)

# CAPÍTULO 7
## ORIENTAÇÃO A OBJETOS

Considere um programa para um banco financeiro. É fácil perceber que uma entidade importante para o
nosso  sistema  será  uma  conta.  

Primeiramente  suponha  que  você  tem  uma  conta  nesse  banco  com  as
seguintes  características:  titular,  número,  saldo  e  limite.  Vamos  começar  inicializando  essas
características:

In [0]:
numero = '123-4'
titular = "João"
saldo = 120.0
limite = 1000.0

E se a necessidade de representar mais de uma conta surgir? Vamos criar mais uma:

In [0]:
numero1 = '123-4'
titular1 = "João"
saldo1 = 120.0
limite1 = 1000.0

In [0]:
numero2 = '123-5'
titular2 = "José"
saldo2 = 200.0
limite2 = 1000.0

Nosso  banco  pode  vir  a  crescer  e  ter  milhares  de  contas  e,  da  maneira  que  está  o  programa,  seria
muito trabalhoso dar manutenção.

E como utilizar os dados de uma determinada conta em outro arquivo? Podemos utilizar a estrutura do dicionário que aprendemos anteriormente e agrupar essas características. Isso vai ajudar a acessar os
dados de uma conta específica:

In [0]:
conta = {"numero": '123-4', "titular": "João", "saldo": 120.0, "limite": 1000.0}

Agora é possível acessar os dados de uma conta pelo nome da chave:

In [0]:
conta['numero']

In [0]:
conta['titular']

Para criar uma segunda conta, crie outro dicionário:

In [0]:
conta2 = {"numero": '123-5', "titular": "José", "saldo": 200.0, "limite": 1000.0}

Avançamos  em  agrupar  os  dados  de  uma  conta,  mas  ainda  precisamos  repetir  seguidamente  essa
linha de código a cada conta criada. Podemos isolar esse código em uma função responsável por criar uma conta:

In [0]:
def cria_conta():
    conta = {"numero": '123-4', "titular": "João", "saldo": 120.0, "limite": 1000.0}
    return conta

Mas ainda não é o ideal já que queremos criar contas com outros valores e tornar a criação dinâmica.
Vamos, então, receber esse valores como parâmetros da função e por fim retornamos a conta:

In [0]:
def cria_conta(numero, titular, saldo, limite):
    conta = {"numero": numero, "titular": titular, "saldo": saldo, "limite": limite}
    return conta

Desta maneira é possível criar várias contas com dados diferentes:

In [0]:
conta1 = cria_conta('123-4', 'João', 120.0, 1000.0)

In [0]:
conta2 = cria_conta('123-5', 'José', 200.0, 1000.0)

Para acessar o número de cada uma delas, fazemos:

In [0]:
conta1['numero']

In [0]:
conta2['numero']

### 7.1 FUNCIONALIDADES
Já  descrevemos  as  características  de  uma  conta  e  nosso  próximo  passo  será  descrever  suas
funcionalidades. O que fazemos com uma conta? Ora, podemos depositar um valor em uma conta, por
exemplo. Vamos criar uma função para representar esta funcionalidade. Além do valor a ser depositado,
precisamos saber qual conta receberá este valor:

In [0]:
def deposita(conta, valor):
    conta['saldo'] = conta['saldo'] + valor

Veja  que  estamos  repetindo  conta`['saldo']`  duas  vezes  nessa  linha  de  código.  O  Python  permite escrever a mesma coisa de uma maneira mais elegante utilizando o `+=`:

In [0]:
def deposita(conta, valor):
    conta['saldo'] += valor

Podemos fazer algo semelhante com a função  `saca()`:

In [0]:
def saca(conta, valor):
    conta['saldo'] -= valor

Antes de testar essas funcionalidades, crie outra que mostra o extrato da conta:

In [0]:
def extrato(conta):
    print("numero: {} \nsaldo: {}".format(conta['numero'], conta['saldo']))

O extrato imprime as informações da conta utilizando a função   `print()`. Agora podemos testar o código:

In [0]:
conta = cria_conta('123-4', 'João', 120.0, 1000.0)

In [0]:
deposita(conta, 15.0)

In [0]:
extrato(conta)

In [0]:
saca(conta, 20.0)

In [0]:
extrato(conta)

Ótimo! Nosso código funcionou como o esperado. Aplicamos algumas funções como   `deposita()` e  `saca()`  e ao final pudermos checar o saldo final com a função  `extrato()`.

### 7.2 EXERCÍCIO: CRIANDO UMA CONTA
1.  Crie uma pasta chamada  `oo`  em sua workspace e crie um arquivo chamado `teste_conta.py`

2.  Crie a função chamada  `cria_conta()` , que recebe como argumento  `numero` ,  `titular` ,  `saldo`  e `limite`:

        def cria_conta(numero, titular, saldo, limite):

3.  Dentro de   `cria_conta()` ,  crie  uma  variável  do  tipo  dicionário  chamada   `conta`   com  as  chaves
recebendo os valores dos parâmetros ( `numero` ,  `titular` ,   `saldo`  e   `limite` ) e ao final retorne a
 `conta`:
        def cria_conta(numero, titular, saldo, limite):
        conta = {"numero": numero, "titular": titular, "saldo": saldo, "limite": limite}
        return conta
4.  Crie  uma  função  chamada   `deposita()`   no  mesmo  arquivo   `teste_conta.py`   que  recebe  como argumento uma  conta  e um  valor. Dentro da função adicione o  valor  ao  saldo  da conta:
        def deposita(conta, valor):
        conta['saldo'] += valor

5.  Crie  outra  função  chamada   `saca()`   que  recebe  como  argumento  uma   conta   e  um   valor. 
Dentro da função subtraia o  valor  do  saldo  da conta:
        def saca(conta, valor):
        conta['saldo'] -= valor

6.  E  por  fim,  crie  uma  função  chamada   `extrato()` ,  que  recebe  como  argumento  uma   conta   e imprime o  numero  e o  saldo:
        def extrato(conta):
        print("numero: {} \nsaldo: {}".format(conta['numero'], conta['saldo']))

7.  Navegue  até  a  pasta   `oo`   pelo  terminal,  abra  o  console  do  Python3,  importe  o  script  e  testes  as
funcionalidades:
 
        from teste_conta import cria_conta, deposita, saca, extrato
        conta = cria_conta('123-7', 'João', 500.0, 1000.0)
        deposita(conta, 50.0)
        extrato(conta)
        saca(conta, 20.0)
        extrato(conta)

8.  (Opcional) Acrescente uma documentação para o seu módulo   `teste_conta.py`  e utilize a função
 `help()`  para testá-la.

Neste exercício criamos uma conta e juntamos suas características (número, titular, limite, saldo) e
funcionalidades  (sacar,  depositar,  tirar  extrato)  num  mesmo  arquivo.  Mas  o  que  fizemos  até  agora  foi
baseado no conhecimento procedural que tínhamos do Python3.

Por mais que tenhamos agrupado os dados de uma conta, essa ligação é frágil no mundo procedural e se mostra limitada. Precisamos pensar sobre o que escrevemos para não errar. O paradigma orientado a objetos vem para sanar essa e outras fragilidades do paradigma procedural que veremos a seguir.

### 7.3 CLASSES E OBJETOS
Ninguém deveria ter acesso ao saldo diretamente. Além disso, nada nos obriga a validar esse valor e podemos  esquecer  disso  cada  vez  que  utilizá-lo.  Nosso  programa  deveria  obrigar  o  uso  das  funções `saca()`  e  `deposita()`  para alterar o saldo e não permitir alterar o valor diretamente:

In [0]:
conta3['saldo'] = 100000000.0

ou então:

In [0]:
conta3['saldo'] = -3000.0

Devemos  manipular  os  dados  através  das  funcionalidades   `saca()`   e   `deposita()`   e  proteger  os dados da conta. Pensando no mundo real, ninguém pode modificar o saldo de sua conta quando quiser, a não ser quando vamos fazer um saque ou um depósito. A mesma coisa deve acontecer aqui.

Para  isso,  vamos  primeiro  entender  o  que  é    classe    e    objeto  ,  conceitos  importantes  do
paradigma orientado a objetos e depois veremos como isso funciona na prática.

Quando  preparamos  um  bolo,  geralmente,  seguimos  uma  receita  que  define  os  ingredientes  e  o modo  de  preparação.  A  nossa  conta  é  um  objeto  concreto  assim  como  o  bolo  que  também  precisa  de uma  receita  pré-definida.  E  a  "receita"  no  mundo  OO (Orientação a Objetos)  recebe  o  nome  de  classe.  Ou  seja,  antes  de
criarmos um objeto definiremos uma classe.

Outra analogia que podemos fazer é entre o projeto de uma casa (a planta da casa) e a casa em si. O projeto  é  a  classe  e  a  casa,  construída  a  partir  desta  planta,  é  o  objeto.  O  projeto  da  conta,  isto  é,  a
definição da conta, é a classe. O que podemos construir (instanciar) a partir dessa classe, as contas de verdade, damos o nome de objetos.

Pode  parecer  óbvio,  mas  a  dificuldade  inicial  do  paradigma  da  orientação  a  objetos  é  justamente
saber  distinguir  o  que  é  classe  e  o  que  é  objeto.  

É  comum  o  iniciante  utilizar,  obviamente  de  forma
errada, essas duas palavras como sinônimos.

O próximo passo será criar nossa classe `Conta`  dentro de um novo arquivo Python, que receberá o nome de `conta.py`. Criar uma classe em Python é extremamente simples em termos de sintaxe. Vamos começar  criando  uma  classe  vazia.  Depois  criaremos  uma  instância,  um  objeto  dessa  classe,  e
utilizaremos a função  `type()`  para analisar o resultado:

In [0]:
class Conta:
    pass

In [0]:
conta = Conta()

In [0]:
type(conta)

__main__.Conta

Vemos porque estamos utilizando o modo interativo pelo terminal e o módulo onde se encontra a classe  `Conta`  é  `conta`. Agora temos uma classe  `Conta`.

Como  Python  é  uma  linguagem  dinâmica,  podemos  modificar  esse  objeto   `conta`   em  tempo  de execução. Por exemplo, podemos acrescentar atributos a ele:

In [0]:
conta.titular = "João"

In [0]:
print(conta.titular)

In [0]:
conta.saldo = 120.0

In [0]:
print(conta.saldo)

Mas  o  problema  do  código  é  que  ainda  não  garantimos  que  toda  instância  de   `Conta`   tenha  um atributo   `titular`  ou   `saldo`.  Portanto  queremos  uma  forma  padronizada  da  conta  de  maneira  que possamos criar objetos com determinadas configurações iniciais.

Em linguagens orientadas a objetos existe uma maneira padronizada de criar atributos de um objeto. Geralmente  fazemos  isso  através  de  uma  função  construtora  -  algo  parecido  com  nossa  função
 `cria_conta()`  do exercício anterior.

### 7.4 CONSTRUTOR
Em Python, alguns nomes de métodos estão reservados para o uso da própria linguagem. Um desses métodos  é  o   `__init__()`   que  vai  inicializar  o  objeto.  Seu  primeiro  parâmetro,  assim  como  todo método de instância, é a própria instância. Por convenção chamamos este argumento de `self`.  Vejamos um exemplo:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = titular
        self.saldo = saldo
        self.limite = limite

Agora,  quando  uma  classe  é  criada,  todos  os  seus  atributos  serão  inicializados  pelo  método `__init__()`.

Apesar de muitos programadores chamarem este método de construtor, ele não cria um objeto  `conta`.  Existe  outro  método,  o    `__new__()`  que  é  chamado  antes  do    `__init_()`    pelo interpretador do Python. O método  `__new__()`  é realmente o construtor e é quem realmente cria uma instância de  `Conta` . O método  `__init__()`  é responsável por inicializar o objeto, tanto é que já recebe
a própria instância (`self`) criada pelo construtor como argumento. E dessa maneira garantimos que toda instância de uma  `Conta`  tenha os atributos que definimos.

Agora, se executarmos a linha de código abaixo, vai acusar um erro:

In [0]:
conta = Conta()

O erro acusa a falta de 4 argumentos na hora de criar uma  `Conta` . A classe  `Conta`  agora nos obriga a passar 4 atributos (`numero`, `titular`, `saldo` e `limite`) para criar uma conta:

In [0]:
conta = Conta('123-4', 'João', 120.0, 1000.0)

Veja  que  em  nenhum  momento  chamamos  o  método   `__init__()`.  Quem  está  fazendo  isso  por
debaixo dos panos é o próprio Python quando executa   `conta  =  Conta()` .  Não  só,  como  vimos,  ele
chama  o  método   `__new__()`   que  devolve  um  instância  do  objeto  e  em  seguida  chama  o  método `__init__()` toda  vez  que  criamos  uma  conta.  Podemos  ver  isto  funcionando  imprimindo  uma mensagem dentro do método  `__init__()`:

In [0]:
def __init__(self, titular, numero, saldo, limite):
    print("inicializando uma conta")
    self.titular = titular
    self.numero = numero
    self.saldo = saldo
    self.limite = limite

e testar novamente:

In [0]:
conta = Conta('123-4', 'João', 120.0, 1000.0)

Ao  criar  uma   `Conta`,  estamos  pedindo  para  o  Python  criar  uma  nova  instância  de   `Conta`   na memória, ou seja, o Python alocará memória suficiente para guardar todas as informações da   `Conta` dentro  da  memória  do  programa.  O   `__new__()`,  portanto,  devolve  uma  referência,  uma  seta  que aponta para o objeto em memória e é guardada na variável  `conta`.

Para manipularmos nosso objeto  `conta`  e acessar seus atributos utilizamos o operador `.` (ponto):

In [0]:
conta.titular

In [0]:
conta.saldo

Como o `self` é a referência do objeto, ele chama  `self.titular`  e  `self.saldo`  da classe  `Conta`.

Agora,  além  de  funcionar  como  esperado,  nosso  código  não  permite  criar  uma  conta  sem  os atributos que definimos anteriormente. Discuta com seus colegas e instrutor as vantagens da orientação a objetos até aqui.

### 7.5 MÉTODOS
Como  vimos,  além  dos  atributos,  nossa  conta  deve  possuir  funcionalidades.  Criamos  as  funções   `saca() ,   `deposita()`   e   `extrato()`.  No  paradigma  orientado  a
objetos as funcionalidades de um objeto são chamados de métodos - do ponto de vista do código, são as funções dentro de uma classe.

Vamos criar o método  `deposita()`  na classe  `Conta`. Aqui, assim como o método  `__init__()`, o método  `deposita()` deve receber a instância do objeto (`self`) além do valor a ser depositado:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = titular
        self.saldo = saldo
        self.limite = limite

    def deposita(self, valor):
        self.saldo += valor

Isso  acontece  porque  o  método  precisa  saber  qual  objeto  `conta`  ele  deve  manipular,  qual  conta  vai
depositar um determinado valor - e podemos ter muitas contas criadas no nosso sistema.

Utilizamos o operador `.` (ponto) através do objeto  `conta`  para chamar o método  `deposita()`:

In [0]:
conta.deposita(20.0)

O interpretador, ao ler esse código, associa o objeto   `conta`  ao argumento   `self`  do método - note que não precisamos passar a  conta  como argumento, isso é feito por debaixo dos panos pelo Python.

Faremos o mesmo para os métodos  `saca()`  e  `extrato()`:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = titular
        self.saldo = saldo
        self.limite = limite
    def deposita(self, valor):
        self.saldo += valor


    def saca(self, valor):
        self.saldo -= valor
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))

Agora vamos testar nossos métodos:

In [0]:
conta = Conta('123-4', 'João', 120.0, 1000.0)

In [0]:
conta.deposita(20.0)

In [0]:
conta.extrato()

In [0]:
conta.saca(15)

In [0]:
conta.extrato()

O  saldo  inicial  era  de  120  reais.  Depositamos  20  reais,  sacamos  15  reais  e  tiramos  o  extrato  que
resultou em 125 reais.

Por fim, o código de nossa  Conta  vai ficar assim:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = tituar
        self.saldo = saldo
        self.limite = limite
    def deposita(self, valor):
        self.saldo += valor
    def saca(self, valor):
        self.saldo -= valor
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))

### 7.6 MÉTODOS COM RETORNO
Em outras linguagens como C++ e Java, um método sempre tem que definir o que retorna, nem que defina  que  não  há  retorno.  Como  vimos  no  capítulo  sobre  funções,  no  Python  isso  não  é  necessário  mas  podemos  retornar  algo  no  método   `saca()` ,  por  exemplo,  indicando  se  a  operação  foi  bem sucedida ou não. Neste caso podemos retornar um valor booleano:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = tituar
        self.saldo = saldo
        self.limite = limite
    def deposita(self, valor):
        self.saldo += valor

    # método alterado
    def saca(self, valor):
    if (self.saldo < valor):
        return False
    else:
        self.saldo -= valor
        return True
    
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))



Veja que a declaração do método não mudou mas agora ele nos retorna algo (um boolean). A palavra chave `return` indica que o método vai terminar ali, retornando tal informação.

Exemplo de uso:

In [0]:
minha_conta.saldo = 1000
consegui = minha_conta.saca(2000)
if(consegui):
    print(“consegui sacar”)
else:
    print(“não consegui sacar”)

Ou então, podemos eliminar a variável temporária, se desejado:

In [0]:
minha_conta.saldo = 1000
if(minha_conta.saca(2000)):
    print(“consegui sacar”)
else:
    print(“não consegui sacar”)

Mais adiante, veremos que algumas vezes é mais interessante lançar uma exceção (exception) nesses casos.

### 7.7 OBJETOS SÃO ACESSADOS POR REFERÊNCIA
O programa pode manter na memória não apenas uma  `Conta`, mas mais de uma:

In [0]:
minha_conta = Conta()
minha_conta.saldo = 1000

In [0]:
meu_sonho = Conta()
meu_sonho.saldo = 1500000

Quando  criamos  uma  variável  para  associar  a  um objeto,  na  verdade,  essa  variável  não  guarda  o objeto, e sim uma maneira de acessá-lo, chamada de referência (o `self`).

In [0]:
c1 = Conta()

Ao  fazer  isso,  já  sabemos  que  o  Python  está  chamando  os  métodos  mágicos `__new__()` e
 `__init__()`  que são responsáveis por construir e iniciar um objeto do tipo  `Conta`.

O correto é dizer que  `c1`  se refere a um objeto. Não é correto dizer que  `c1`  é um objeto, pois  `c1`  é uma  variável  referência,  apesar  de,  depois  de  um  tempo,  os  programadores  falarem  “tenho  um  objeto `c1`  do tipo   `Conta`”, mas apenas para encurtar a frase “Tenho uma referência   `c1` a  um  objeto  tipo `Conta`”.

Vamos analisar o código abaixo:

In [0]:
c1 = Conta('123-4', 'João', 120.0, 1000.0) 
c2 = c1
c2.saldo

In [0]:
c1.deposita(100.0)
c1.saldo

In [0]:
c2.deposita(30.0) 


In [0]:
c1.saldo
c2.saldo

O que aconteceu aqui? O operador `=` copia o valor de uma variável. Mas qual é o valor da variável `c1` ?  É  o  objeto?  Não.  Na  verdade,  o  valor  guardado  é  a  referência  (endereço)  de  onde  o  objeto  se
encontra na memória principal.

Ao fazer   `c2  =  c1` ,   `c2`   passa  a  fazer  referência  para  o  mesmo  objeto  que   `c1`   referencia  nesse instante. Quando utilizamos   `c1`  ou   `c2` , neste código, estamos nos referindo ao MESMO objeto – são
duas referências que apontam para o mesmo objeto.

Podemos notar isso através da função interna  `id()`  que retorna a referência de um objeto:

In [0]:
id(c1)

In [0]:
id(c2)

Internamente,  `c1`  e  `c2`  vão guardar um número que identifica em que posição da memória aquela `Conta`  se encontra. Dessa maneira, ao utilizarmos o `.` (ponto) para navegar, o Python vai acessar a `Conta`   que  se  encontra  naquela  posição  de  memória,  e  não  uma  outra  conta.

Para  quem  conhece,  é parecido com um ponteiro, porém você não pode manipulá-lo.

Outra  maneira  de  notar  esse  comportamento  é  que  o  interpretador  Python  chamou  os  métodos `__new__()`   e   `__init__()`    apenas  uma  vez  (na  linha   `c1  =  Conta('123-4',  'João',  120.0, 1000.0)` ), então só pode haver um objeto   Conta  na memória. Compará-las com o operador `==` vai nos retornar  True , pois o valor que elas carregam é o mesmo:

In [0]:
id(c1) == id(c2)

In [0]:
c1 == c2

Podemos então ver outra situação:

In [0]:
c1 = Conta("123-4", "Python", 500.0, 1000.0)
c2 = Conta("123-4", "Python", 500.0, 1000.0)

In [0]:
if(c1 == c2):
    print("contas iguais")
else:
    print("contas diferentes")

O operador `==` compara o conteúdo das variáveis, mas essas variáveis não guardam o objeto, e sim o endereço em que ele se encontra. Como em cada uma dessas variáveis guardamos duas contas criadas diferentemente, elas estão em espaços diferentes da memória, o que faz o teste no `if`  valer   `False`. As contas  podem  ser  equivalentes  no  nosso critério  de  igualdade,  porém  elas  não  são  o  mesmo  objeto.

Quando se trata de objetos, pode ficar mais fácil pensar que o `==` compara se os objetos (referências, na verdade) são o mesmo, e não se possuem valores iguais.

Para  saber  se  dois  objetos  têm  o  mesmo  conteúdo, você  precisa  comparar  atributo  por  atributo.

Futuramente, veremos uma solução mais elegante para isso também.

### 7.8 MÉTODO TRANSFERE
E  a  funcionalidade  que  transfere  dinheiro  entre  duas  contas?  Podemos  ficar  tentados  a  criar  um método que recebe dois parâmetros:   `conta1`  e   `conta2`  do tipo   `Conta`. Cuidado: já sabemos que os métodos de nossa classe   `Conta`  sempre recebem a referência, o `self` - portanto o método recebe apenas um parâmetro do tipo  `Conta` , a conta destino (além do valor):

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = tituar
        self.saldo = saldo
        self.limite = limite
    def deposita(self, valor):
        self.saldo += valor
    def saca(self, valor):
    if (self.saldo < valor):
        return False
    else:
        self.saldo -= valor
        return True
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))

    # método adicionado
    def transfere(self, destino, valor):
        self.saldo -= valor
        destino.saldo += valor

Para  deixar  o  código  mais  robusto,  poderíamos  verificar  se  a  conta  possui  a  quantidade  a  ser
transferida disponível. Para ficar ainda mais interessante, você pode chamar os métodos `deposita()`  e `saca()`  já existentes para fazer essa tarefa:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite):
        self.numero = numero
        self.titular = tituar
        self.saldo = saldo
        self.limite = limite
    def deposita(self, valor):
        self.saldo += valor
    def saca(self, valor):
    if (self.saldo < valor):
        return False
    else:
        self.saldo -= valor
        return True
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))

    # método alterado
    def transfere(self, destino, valor):
        retirou = self.saca(valor)
        if (retirou == False):
            return False
        else:
            destino.deposita(valor)
            return True

Quando passamos uma `Conta`  como argumento, o que será que acontece na memória? Será que o objeto é clonado?

No Python, a passagem de parâmetro funciona como uma simples atribuição como no uso do `=`.

Então, esse parâmetro vai copiar o valor da variável do tipo   `Conta`  que for passado como argumento para  a  variável   destino.  E  qual  é  o  valor  de  uma  variável  dessas?  Seu  valor  é  um  endereço,  uma referência, nunca um objeto. Por isso não há cópia de objetos aqui.

Esse último código poderia ser escrito com uma sintaxe muito sucinta. Como?

    TRANSFERE PARA

Perceba  que  o  nome  deste  método  poderia  ser    `transfere_para()`    ao  invés  de  só `transfere()`. A chamada do método fica muito mais natural, é possível ler a frase em português que ela tem um sentido:

In [0]:
conta1.transfere_para(conta2, 50.0):

A leitura deste código seria "`conta1` transfere para `conta2` `50` reais".

### 7.9 CONTINUANDO COM ATRIBUTOS
Os  atributos  de  uma  classe  podem  receber  um  valor padrão  -  assim  como  os  argumentos  de  uma função.

Nosso  banco  pode  ter  um  valor  de  limite  padrão  para todas as contas e apenas em casos específicos pode atribuir um valor limite diferente.

Para  aplicarmos  essa  regra  de  negócio,  podemos atribuir  um  valor  padrão  ao  limite,  por  exemplo, 1000.0 reais:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite=1000.0):
        self.numero = numero
        self.titular = titular
        self.saldo = saldo
        self.limite = limite

E podemos inicializar uma conta:

In [0]:
conta = Conta('123-4', 'joão', 120.0)

Veja que agora não somos obrigados a passar o valor do limite  já que ele possui um valor padrão de 1000.0 e podemos acessá-lo pela conta:

In [0]:
conta.limite

Quando  declaramos  as  variáveis  na  classe   `Conta`, aprendemos  que  podemos  atribuir  um  valor
padrão  para  cada  uma  delas.  Imagine  que  comecemos  a  aumentar  nossa  classe   `Conta`   e  adicionar
nome,  sobrenome  e  cpf  do  titular  da  conta.

Começaríamos  a  ter  muitos  atributos...  e,  se  você  pensar direito, uma   `Conta`  não tem nome, nem sobrenome nem cpf, quem tem esses atributos é um cliente.

Então  podemos  criar  uma  nova  classe  e  fazer  uma  agregação  -  agregar  um  cliente  a  nossa  conta.

Portanto, nossa classe  `Conta`  tem um  `Cliente`.

O atributos de uma   `Conta`  também podem ser referências para outras classes. Suponha a seguinte classe  `Cliente`:

In [0]:
class Cliente:
    def __init__(self, nome, sobrenome, cpf):
        self.nome = nome
        self.sobrenome = sobrenome
        self.cpf = cpf

In [0]:
class Conta:
    def __init__(self, numero, cliente, saldo, limite):
        self.numero = numero
        self.titular = cliente
        self.saldo = saldo
        self.limite = limite

E quando criarmos um  `Conta`, precisamos passar um  `Cliente` como  titular :

In [0]:
cliente = Cliente('João', 'Oliveira', '1111111111-1')

In [0]:
minha_conta = Conta('123-4', cliente, 120.0, 1000.0)

Aqui aconteceu uma atribuição, o valor da variável `cliente`  é copiado para o atributo  `titular`  do objeto  ao  qual   `minha_conta` se  refere. Em  outras  palavras, `minha_conta`   tem  uma  referência  ao mesmo  `Cliente`  que  `cliente`  se refere, e pode ser acessado através de  `minha_conta.titular`.

Você pode realmente navegar sobre toda estrutura de informação, sempre usando o ponto:

In [0]:
minha_conta.titular

Veja que a saída é a referência a um objeto do tipo `Cliente`, mas podemos acessar seus atributos de uma forma mais direta e até mais elegante:

In [0]:
minha_conta.titular.nome

### 7.10 TUDO É OBJETO
Python é uma linguagem totalmente orientada a objetos. Tudo em Python é um objeto! Sempre que utilizamos uma função ou método que recebe parâmetros estamos passando objetos como argumentos.

Não  é  diferente  com  nossas  classes.  Quando  uma  conta  recebe  um  cliente  como  titular,  ele  está recebendo uma instância de `Cliente` , ou seja, um objeto.

O mesmo acontece com  `numero` ,  `saldo`  e   `limite`. Strings e números são classes no Python. Por este  motivo  que  aparece  a  palavra  class  quando  pedimos  para  o  Python  nos  devolver  o  tipo  de  uma variável através da função `type`:

In [0]:
type(conta.numero)

In [0]:
type(conta.saldo)

In [0]:
type(conta.titular)

Um sistema orientado a objetos é um grande conjunto de classes que vai se comunicar, delegando responsabilidades para quem for mais apto a realizar determinada tarefa. A classe `Banco`   usa  a  classe `Conta`   que  usa  a  classe   `Cliente`,  que  usa  a  classe `Endereco` ,  etc... Dizemos  que  esses  objetos colaboram,  trocando  mensagens  entre  si.  Por  isso  acabamos  tendo  muitas  classes  em  nosso  sistema,  e elas costumam ter um tamanho relativamente curto.

### 7.11 COMPOSIÇÃO
Fizemos,  no  ponto  anterior,  uma  agregação.  Agora  nossa  classe   Conta   tem  um   Cliente   e associamos estas  duas  classes.  Mas  nossa  classe   `Cliente` existe  independente  da  classe `Conta`.

Suponha  agora  que  nossa   `Conta`   possua  um  histórico,  contendo  a  data  de  abertura  da  conta  e  suas transações. Podemos criar uma classe para representar o histórico, como no exemplo abaixo:

In [0]:
import datetime
class Historico:
    def __init__(self):
        self.data_abertura = datetime.datetime.today()
        self.transacoes = []
    def imprime(self):
        print("data abertura: {}".format(self.data_abertura))
        print("transações: ")
        for t in self.transacoes:
            print("-", t)

Agora precisamos modificar nossa classe  `Conta`  de modo que ela tenha um  `Historico` . Mas aqui, diferente  da  relação  do  cliente  com  uma  conta,  a  existência  de  um histórico  depende  da  existência  de uma  `Conta`:

In [0]:
class Conta:
    def __init__(self, numero, cliente, saldo, limite=1000.0):
        self.numero = numero
        self.cliente = cliente
        self.saldo = saldo
        self.limite = limite
        self.historico = Historico()

E podemos, em cada método para manipular uma  `Conta`, acrescentar a operação nas transações de seu  `Historico`:

In [0]:
class Conta:
    def __init__(self, numero, cliente, saldo, limite=1000.0):
        self.numero = numero
        self.cliente = cliente
        self.saldo = saldo
        self.limite = limite
        self.historico = Historico()

    # código adicionado
    def deposita(self, valor):
        self.saldo += valor
        self.historico.transacoes.append("depósito de {}".format(valor))
    def saca(self, valor):
        if (self.saldo < valor):
            return False
        else:
            self.saldo -= valor
            self.historico.transacoes.append("saque de {}".format(valor))
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))
        self.historico.transacoes.append("tirou extrato - saldo 
        de {}".format(self.saldo))
    def transfere_para(self, destino, valor):
        retirou = self.saca(valor)
        if (retirou == False):
            return False
        else:
            destino.deposita(valor)
            self.historico.transacoes.append("transferencia de {} 
            para conta {}".format(valor, destino.numero))
            return True

E testamos:

In [0]:
cliente1 = Cliente('João', 'Oliveira', '11111111111-11')
cliente2 = Cliente('José', 'Azevedo', '222222222-22')
conta1 = Conta('123-4', cliente1, 1000.0)
conta2 = Conta('123-5', cliente2, 1000.0)
conta1.deposita(100.0)
conta1.saca(50.0)
conta1.transfere_para(conta2, 200.0)
conta1.extrato

In [0]:
conta1.historico.imprime()

In [0]:
conta2.historico.imprime()

Quando a existência de uma classe depende de outra classe, como é a relação da classe `Histórico` com a classe `Conta`, dizemos que a classe `Historico` compõe a classe `Conta`.

Esta  associação  chamamos Composição.

Mas, e se dentro da nossa   `Conta`  não colocássemos   `self.historico = Historico()`  e tentasse acessá-lo diretamente?

Faz algum sentido fazer `historico = Historico()`?

Quando o objeto é inicializado, ele vai receber o valor default que definimos na classe:

In [0]:
class Conta:
    def __init__(self, numero, cliente, saldo, limite=1000.0):
        self.numero = numero
        self.cliente = cliente
        self.saldo = saldo
        self.limite = limite
        # código alterado (?)
        self.historico = Historico()

    def deposita(self, valor):
        self.saldo += valor
        self.historico.transacoes.append("depósito de {}".format(valor))
    def saca(self, valor):
        if (self.saldo < valor):
            return False
        else:
            self.saldo -= valor
            self.historico.transacoes.append("saque de {}".format(valor))
    def extrato(self):
        print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))
        self.historico.transacoes.append("tirou extrato - saldo 
        de {}".format(self.saldo))
    def transfere_para(self, destino, valor):
        retirou = self.saca(valor)
        if (retirou == False):
            return False
        else:
            destino.deposita(valor)
            self.historico.transacoes.append("transferencia de {} 
            para conta {}".format(valor, destino.numero))
            return True        

Com esse código, toda nova  `Conta`  criada já terá um novo `Historico` associado, sem necessidade de instanciá-lo logo em seguida da instanciação de uma  `Conta`

Atenção: para quem não está acostumado com referências, pode ser bastante confuso pensar sempre em  como  os  objetos  estão  na  memória  para  poder  tirar  as  conclusões  de  o  que  ocorrerá  ao  executar determinado  código,  por  mais  simples  que  ele  seja.  Com  o  tempo,  você  adquire  a  habilidade  de rapidamente  saber  o  efeito  de  atrelar  as  referências,  sem  ter  de  gastar  muito  tempo  para  isso.  É importante, nesse começo, você estar sempre pensando no estado da memória. E realmente lembrar que,
no Python “uma variável nunca carrega um objeto, e sim uma referência para ele” facilita muito.

### 7.12 PARA SABER MAIS: OUTROS MÉTODOS DE UMA CLASSE
O interpretador adiciona alguns atributos especiais somente para leitura a vários tipos de objetos de uma classe e um deles é o  `__dict__`.

Isso  acontece  porque  a  classe   `Conta`   possui  alguns  métodos,  dentre  eles  o   `__init__()`   e  o  `__new__()`   que  são  chamados  para  criar  e  inicializar  um  objeto  desta  classe,  respectivamente.  Caso você queira saber quais outros métodos são implementados pela classe   `Conta`  você pode usar a função embutida  `dir()`  que vai listar todos métodos e atributos que a classe possui.



In [0]:
dir(Conta)

Dessa lista, já conhecemos o   `__init__()`, o   `__new__()`  e os métodos e atributos que definimos quando construímos a classe   `Conta`.  Na  verdade,  quando  usamos  a  função   `dir()`,  o  interpretador chama o atributo   `__dir__`  dessa lista. Um outro atributo bastante útil é o   `__dict__`  que retorna um dicionário com os atributos da classe:

In [0]:
cliente = Cliente('João', 'Oliveira', '111111111-11')
conta = Conta('123-4', cliente, 1000.0)
conta.__dict__

Mas  não  é  comum  acessá-lo  dessa  maneira.  Estes métodos  iniciados  e  terminados  com  dois underscores  são  chamados  pelo  interpretador  e  são  conhecidos  como  métodos  mágicos.  Existe  outra função  embutida  do  Python,  a  função   `vars()`,  que  chama  exatamente  o   `__dict__`   de  uma  classe.

Obtemos o mesmo resultado usando `vars(conta)`:



In [0]:
vars(conta)

Repare  que  o   `__dict__`   e  o   `vars()`   retornam exatamente  um  dicionário  de  atributos  de  uma conta  como  tínhamos  modelado  no  início  deste  capítulo.

Portanto,  nossas  classes  utilizam  dicionários para armazenar informações da própria classe.

Os demais métodos mágicos estão disponíveis para uso e não utilizaremos por enquanto. Voltaremos a falar deles em um outro momento.

### 7.13 EXERCÍCIO: PRIMEIRA CLASSE PYTHON

1.  Crie um arquivo chamado  conta.py  na pasta  oo  criada no exercício anterior.
2.  Crie a classe Conta sem nenhum atributo e salve o arquivo.

        class Conta:
            pass
3.  Abra o terminal e vá até a pasta onde se encontra o arquivo   conta.py . Abra o console do Python3 no terminal e importe a classe Conta do módulo conta.
        from conta import Conta
4.  Crie  uma  instância  (objeto)  da  classe   Conta   e  utilize  a  função   type()   para  verificar  o  tipo  do
objeto:
        conta = Conta()
        type(conta)
Além disso, crie alguns atributos e tente acessá-los.
1.  Abra  novamente  o  arquivo  conta.py  e  escreva  o método   `__init__()`   recebendo  os  atributos
anteriormente definidos por nós que toda conta deve ter (numero titular, saldo e limite):
        class Conta:
            def __init__(self, numero, titular, saldo, limite):
                self.numero = numero
                self.titular = titular
                self.saldo = saldo
                self.limite = limite
2.  Reinicie  o  Python3  no  terminal  e  importe  novamente  a  classe   Conta   do  módulo  conta  para testarmos nosso código:
        from conta import Conta
3.  Tente criar uma conta sem passar qualquer argumento no construtor:
        conta = Conta()

Note  que  o  interpretador  acusou  um  erro.  O  método   `__init__()`   exige  4  argumentos  `numero`,
`titular`, `saldo` e `limite`.
4.  Agora vamos seguir o exigido pela classe, pela receita de uma conta:
        conta = Conta('123-4', 'João', 120.0, 1000.0)
5.  O interpretador não acusou nenhum erro. Vamos imprimir o  numero  e  titular  da conta:
        conta.numero
        conta.titular
6.  Crie o método   `deposita()`  dentro da classe   `Conta`. Esse método deve receber uma referência do
próprio objeto e o valor a ser adicionado ao saldo da conta.
        def deposita(self, valor):
            self.saldo += valor
7.  Crie o método  saca()  que recebe como argumento uma referência do próprio objeto e o valor a ser
sacado. Esse método subtrairá o valor do saldo da conta.
 def saca(self, valor):
     self.saldo -= valor
8.  Crie  o  método   `extrato()` ,  que  recebe  como  argumento  uma  referência  do  próprio  objeto.  Esse
método imprimirá o saldo da conta:
        def extrato(self):
            print("numero: {} \nsaldo: {}".format(self.numero, self.saldo))
9.  Modifique o método `saca()` fazendo retornar um valor que representa se a operação foi ou não bem sucedida. Lembre que não é permitido sacar um valor menor do que o saldo.
        def saca(self, valor):
            if (self.saldo < valor):
                return False
            else:
                self.saldo -= valor
                return True
10.  Crie o método   `transfere_para()`  que recebe como argumento uma referência do próprio objeto, uma  `Conta`  destino e o valor a ser transferido. Esse método deve sacar o valor do próprio objeto e depositar na conta destino:
        
            def transfere_para(self, destino, valor):
                retirou = self.saca(valor)
                if (retirou == false):
                    return False
                else:
                    destino.deposita(valor)
                    return True

11.  Abra o Python no terminal, importe o módulo `conta`, crie duas contas e teste os métodos criados.

12.  (Opcional)  Crie  uma  classe  para  representar  um  cliente  do  nosso  banco  que  deve  ter    nome  ,
 sobrenome  e  cpf . Instancie uma  Conta  e passe um cliente como   titular  da conta. Modifique
o método   `extrato()`   da  classe   `Conta`   para  imprimir,  além  do  número  e  o  saldo,  os  dados  do
cliente. Podemos criar uma  `Conta`  sem um  `Cliente` ? E um  `Cliente`  sem uma  `Conta`?
13.  (Opcional)  Crie  uma  classe  que  represente  uma  data,  com  dia,  mês  e  ano.Crie  um  atributo
 `data_abertura`  na classe  `Conta`. Crie uma nova conta e faça testes no console do Python.

14.  (Desafio)  Crie  uma  classe   `Historico`   que  represente  o  histórico  de  uma   `Conta`   seguindo  o
exemplo da apostila. Faça testes no console do Python criando algumas contas, fazendo operações e
por último mostrando o histórico de transações de uma  `Conta`. Faz sentido criar um objeto do tipo
 `Historico`  sem uma  `Conta`? 

Agora,  além  de  funcionar  como  esperado,  nosso  código  não  permite  criar  uma  conta  sem  os
atributos que definimos anteriormente. Discuta com seus colegas e instrutor as vantagens da orientação
a objetos até aqui.

# CAPÍTULO 8
## MODIFICADORES DE ACESSO E MÉTODOS DE CLASSE

Um dos problemas mais simples que temos no nosso sistema de contas é que o método  `saca()`  permite sacar mesmo que o saldo seja insuficiente. A seguir você pode lembrar como está a classe  `Conta`:

In [0]:
class Conta:
    def __init__(self, numero, titular, saldo, limite=1000.0):
        self.numero = numero
        self.titular = titular    
        self.saldo = saldo
        self.limite = limite
    # outros métodos
    def saca(self, valor):
        this.saldo -= valor

Abrimos o terminal e testamos nosso código:

In [0]:
minha_conta = Conta('123-4', 'joão', 1000.0, 2000.0)
minha_conta.saca(500000)

O limite de saque é ultrapassado. Podemos incluir um  `if`  dentro do método   `saca()`  para evitar a situação  que  resultaria  em  uma  conta  em  estado  inconsistente,  com  seu  saldo  menor  do  que  zero.

Fizemos isso no capítulo de orientação a objetos básica.
Apesar de melhorar bastante, ainda temos um problema mais grave: ninguém garante que o usuário da  classe  vai  sempre  utilizar  o  método  para  alterar  o  saldo  da  conta.  O  código  a  seguir  altera  o  saldo diretamente:

In [0]:
minha_conta = Conta('123-4', 'João', 1000.0)
minha_conta.saldo = -200

Como evitar isso? Uma ideia simples seria testar se não estamos sacando um valor maior que o saldo toda vez que formos alterá-lo.

In [0]:
minha_conta = Conta('123-4', 'joão', 1000.0)
novo_saldo = -200
    if(novo_saldo < 0):
        print("saldo inválido")
    else:
        minha_conta.saldo = novo_saldo


Esse código iria se repetir ao longo de toda nossa aplicação e, pior, alguém pode esquecer de fazer
essa comparação em algum momento, deixando a conta em uma situação inconsistente. A melhor forma
de resolver isso seria forçar quem usa a classe   Conta  a  invocar o método   saca()   e  não  permitir  o
acesso direto ao atributo.
Em linguagens como Java e C# basta declarar que os atributos não podem ser acessados de fora da
classe  utilizando  a  palavra  chave  private.  Em  orientação  a  objetos,  é  prática  quase  que  obrigatória
proteger seus atributos com private. Cada classe é responsável por controlar seus atributos, portanto ela
deve julgar se aquele novo valor é válido ou não. E esta validação não deve ser controlada por quem está
usando a classe e sim por ela mesma, centralizando essa responsabilidade e facilitando futuras mudanças
no sistema.
O  Python  não  utiliza  o  termo  private,  que  é  um  modificador  de  acesso  e  também  chamado  de
modificador de visibilidade. No Python inserimos dois underscores ('__') ao atributo para adicionar esta
característica:

    class Pessoa:
        def __init__(self, idade):
            self.__idade = idade

Dessa maneira não conseguimos acessar o atributo   idade  de um objeto do tipo   Pessoa  fora da
classe:

    >>> pessoa = Pessoa(20)
    >>> pessoa.idade
    Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    AttributeError: 'Pessoa' object has no attribute 'idade'

O interpretador acusa que o atributo  idade  não existe na classe  Pessoa . Mas isso não garante que
ninguém  possa  acessá-lo.  No  Python  não  existem  atributos  realmente  privados,  ele  apenas  alerta  que
você não deveria tentar acessar este atributo, ou modificá-lo. Para acessá-lo, fazemos:

    >>> p._Pessoa__idade

Ao colocar o prefixo __ no atributo da classe, o Python apenas renomeia `__nome_do_atributo` para
`_nomeda_Classe\_nome_do_atributo`, como fez em `__idade` para `_Pessoa__idade`.  Qualquer  pessoa
que  saiba  que  os  atributos  privados  não  são  realmente  privados,  mas  "desconfigurados",  pode  ler  e
atribuir  um  valor  ao  atributo  "privado"  diretamente.  Mas  fazer   pessoa.`_Pessoa__idade  =  20`  é
considerado má prática e pode acarretar em erros.
Podemos utilizar a função  dir  para ver que o atributo  `_Pessoa__idade`  pertence ao objeto:

    >>> dir(pessoa)
    ['_Pessoa__idade', '__class__', '__delattr__', '__dict__', '__dir__',
    '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__',
    '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', 
    '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', 
    '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__',
    '__weakref__']

Repare que não existe nenhum atributo  __idade  no objeto pessoa. Agora vamos tentar atribuir um
valor para  __idade :

    >>> pessoa.__idade = 25

Epa, será que o Python deveria deixar isso ocorrer? Vamos acessar a variável novamente e ver se a
modificação realmente aconteceu:

    >>> pessoa._Pessoa__idade
    20

O que aconteceu aqui é que o Python criou um novo atributo __idade para o objeto  pessoa  já que
é uma linguagem dinâmica. Vamos utilizar a função  dir  novamente para ver isso:

    >>> dir(pessoa)
    ['_Pessoa__idade', '__class__', '__delattr__', '__dict__', '__dir__',
    '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__',
    '__hash__', '__idade', '__init__', '__init_subclass__', '__le__', '__lt_'
    '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__',
    '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__',
    '__weakref__']
Note que um novo atributo   `__idade`   apareceu,  já  que  foi  inicializado  em  tempo  de  execução  e  é
diferente do   `__idade`  da classe. Isso pode gerar muita confusão e erros! O Python também tem uma
maneira de lidar com este problema através da variável `__slots__` onde definimos um número limitado
de atributos que veremos a seguir.
Nenhum  atributo  é  realmente  privado  em  Python  já  que  podemos  acessá-lo  pelo  seu  nome
'desfigurado'.  Muitos  programadores  Python  não  gostam  dessa  sintaxe  e  preferem  usar  apenas  um
underscore  '_'  para  indicar  quando  um  atributo  deve  ser  protegido.  Ou  seja,  deve  ser  explícita  essa
desconfiguração do nome - feita pelo programador e não pelo interpretador - já que oferece o mesmo
resultado. E argumentam que '__' são obscuros.
O  prefixo  com  apenas  um  underscore  não  tem  significado  para  o  interpretador  quando  usado  em
nome  de  atributos,  mas  entre  programadores  Python  é  uma  convenção  que  deve  ser  respeitada.  O
programador alerta que esse atributo não deve ser acessado diretamente:

    def __init__(self, idade):
        self._idade = idade

Um atributo com apenas um underscore é chamados de protegido, mas quando usado sinaliza que
deve ser tratado como um atributo "privado" e acessá-lo diretamente pode ser perigoso.
As  mesmas  regras  de  acesso  aos  atributos  valem  para  os  métodos.  É  muito  comum,  e  faz  todo
sentido,  que  seus  atributos  sejam  privados  e  quase  todos  seus  métodos  sejam  públicos  (não  é  uma
regra!).  Desta  forma,  toda  conversa  de  um  objeto  com  outro  é  feita  por  troca  de  mensagens,  isto  é,
acessando seus métodos. Algo muito mais educado que mexer diretamente em um atributo que não é
seu.
Melhor  ainda!  O  dia  que  precisarmos  mudar  como  é  realizado  um  saque  na  nossa  classe   Conta ,
adivinhe onde precisaríamos modificar? Apenas no método  saca() , o que faz pleno sentido.

### 8.1 ENCAPSULAMENTO
O que começamos a ver nesse capítulo é a ideia de encapsular, isto é, 'esconder' todos os membros de
uma  classe  (como  vimos  acima),  além  de  esconder  como  funcionam  as  rotinas  (no  caso  métodos)  do
nosso sistema.
Encapsular é fundamental para que seu sistema seja suscetível a mudanças: não precisamos mudar
uma  regra  de  negócio  em  vários  lugares,  mas  sim  em  apenas  um  único  lugar,  já  que  essa  regra  está
encapsulada. O conjunto de métodos públicos de uma classe é também chamado de interface da classe,
pois esta é a única maneira a qual você se comunica com objetos dessa classe.
O  underscore  _  alerta  que  ninguém  deve  modificar,  nem  mesmo  ler,  o  atributo  em  questão.  Com
isso, temos um problema: como fazer para mostrar o saldo de uma   Conta , já que não devemos acessálo para leitura diretamente?
Precisamos então arranjar uma maneira de fazer esse acesso. Sempre que precisamos arrumar uma
maneira  de  fazer  alguma  coisa  com  um  objeto,  utilizamos  métodos!  Vamos  então  criar  um  método,
digamos  pega_saldo() , para realizar essa simples tarefa:

    class Conta:
        # outros métodos
        def pega_saldo(self):
            return self._saldo
            
Para acessarmos o saldo de uma conta, podemos fazer:

    >>> minha_conta = Conta('123-4', 'joão', 1000.0)
    >>> minha_conta.deposita(100)
    >>> minha_conta.pega_saldo()
    1100
    
Para  permitir  o  acesso  aos  atributos  (já  que  eles  são  'protegidos')  de  uma  maneira  controlada,  a
prática  mais  comum  é  criar  dois  métodos,  um  que  retorna  o  valor  e  outro  que  muda  o  valor.  A
convenção para esses métodos em muitas linguagens orientadas a objetos é colocar a palavra get ou set
antes  do  nome  do  atributo.  Por  exemplo,  uma  conta  com  saldo  e  titular  fica  assim,  no  caso  de
desejarmos dar acesso a leitura e escrita a todos os atributos:

In [0]:
class Conta:
    def __init__(self, titular, saldo):
        self._titular = titular
        self._saldo = saldo
    def get_saldo(self):
        return self._saldo
    def set_saldo(self, saldo):
        self._saldo = saldo        
    def get_titular(self):
        return self._titular
    def set_titular(self, titular):
        self._titular = titular

Getters e setters são usados ​​em muitas linguagens de programação orientada a objetos para garantir o
princípio  do  encapsulamento  de  dados.  O  encapsulamento  de  dados  é  visto  como  o  agrupamento  de
dados com os métodos que operam nesses dados. Esses métodos são, obviamente, o getter para recuperar
os dados e o setter para alterar os dados. De acordo com esse princípio, os atributos de uma classe são
tornados privados para ocultá-los e protegê-los de outro código.
Infelizmente,  é  crença  generalizada  que  uma  classe  Python  adequada  deve  encapsular  atributos
privados usando getters e setters. Assim que um desses programadores introduzir um novo atributo, ele
fará  com  que  seja  uma  variável  privada  e  criará  "automaticamente"  um  getter  e  um  setter  para  esses
atributos.
Os  programadores  de  Java  irão  torcer  o  nariz  quando  lerem  o  seguinte:  A  maneira  pythônica  de
introduzir atributos é torná-los públicos. Vamos explicar isso mais tarde. Primeiro, demonstramos no
exemplo a seguir, como podemos projetar uma classe, da mesma maneira usada no Java, com getters e
setters para encapsular um atributo protegido:

In [0]:
class Conta:
    def __init __(self, saldo):
        self._saldo = saldo
    def get_saldo(self):
        retorno self._saldo
    def set_saldo(self, saldo):
        self._saldo = saldo

E podemos ver como trabalhar com essa classe e os métodos:

    >>> conta1 = Conta(200.0)
    >>> conta2 = Conta(300.0)
    >>> conta3 = Conta(-100.0)
    >>> conta1.get_saldo()
    200.0
    >>> conta2.get_saldo()
    300.0
    >>> conta3.set_saldo(conta1.get_saldo() + conta2.get_saldo())
    >>> conta3.get_saldo()
    500.0
    
O  que  você  acha  da  expressão  "conta3.set_saldo(conta1.get_saldo()  +  conta2.get_saldo())"?  É  feio, não é? É muito mais fácil escrever uma expressão como a seguinte:
conta3.saldo = conta1.saldo + conta2.saldo
Tal  atribuição  é  mais  fácil  de  escrever  e,  acima  de  tudo,  mais  fácil  de  ler  do  que  a  expressão  com
getters e setters. Vamos reescrever a classe  Conta  de um modo Pythônico, sem getter e sem setter:

In [0]:
class Conta:
    def __init __(self, saldo):
        self.saldo = saldo

Mas neste caso não há encapsulamento e não seria um problema. Mas o que acontece se quisermos
mudar  a  implementação  no  futuro?  O  leitor  atento  deve  ter  reparado  que  no  exemplo  anterior
declaramos uma variável do tipo   Conta  com saldo negativo e isso não deveria acontecer. Temos que
evitar essa situação e o setter, neste caso, se justifica para acrescentar esta validação:
class Conta:

In [0]:
class Conta:
    def __init __(self, saldo):
        self.saldo = saldo
    def set_saldo(self, saldo):
        if(saldo < 0):
            print("saldo não pode ser negativo")
        else:
            self.saldo = saldo

Podemos abrir o interpretador e testar:

    >>> conta1 = Conta(200.0)
    >>> conta1.saldo
    200.0
    >>> conta2 = Conta(300.0)
    >>> conta2.saldo
    300.0
    >>> conta3 = Conta(100.0)
    >>> conta3.set_saldo(-100.0)
    "saldo não pode ser negativo"
    >>> conta3.saldo
    100.0

Mas há um problema, caso projetemos nossa classe com atributo público e sem métodos quebramos
a interface:

    conta1 = Conta(100.0)
    conta.saldo = -100.0

É por isso que em Java recomenda-se que as pessoas usem somente atributos privados com getters e
setters, para que possam alterar a implementação sem precisar alterar a interface. O Python oferece uma
solução  bastante  parecida  para  este  problema.  A  solução  é  chamada  de  properties.  Mantemos  nossos
atributos protegidos e decoramos nossos métodos com um decorator chamado property.
A classe com uma propriedade fica assim:

In [0]:
class Conta:
    def __init__(self, saldo=0.0):
        self._saldo = saldo
    @property
    def saldo(self):
        return self._saldo    
    @saldo.setter
    def saldo(self, saldo):
        if(self._saldo < 0):
            print("saldo não pode ser negativo")
        else:
            self._saldo = saldo

Um  método  que  é  usado  para  obter  um  valor  (o  getter)  é  decorado  com   @property ,  isto  é,
colocamos  essa  linha  diretamente  acima  da  declaração  do  método  que  recebe  o  nome  do  próprio
atributo. O método que tem que funcionar como setter é decorado com   @saldo.setter .  Se  a  função
tivesse sido chamada de "func", teríamos que anotá-la com  @func.setter .
PARA SABER MAIS: DECORATOR
Um  decorador,  ou  decorator  é  um  padrão  de  projeto  de  software  que  permite  adicionar  um
comportamento  a  um  objeto  já  existente  em  tempo  de  execução,  ou  seja,  agrega  dinamicamente
responsabilidades  adicionais  a  um  objeto.  Esta  solução  traz  uma  flexibilidade  maior,  em  que
podemos adicionar ou remover responsabilidades sem que seja necessário editar o código-fonte.
Um decorador é um objeto invocável, uma função que aceita outra função como parâmetro (a
função  decorada).  O  decorador  pode  realizar  algum  processamento  com  a  função  decorada  e
devolvê-la ou substituí-la por outra função. O property é um decorador que possui métodos extras
como  um  getter  e  um  setter  e  ao  ser  aplicado  a  um  objeto,  retorna  uma  cópia  dele  com  essas
funcionalidades:

    @property
    def foo(self):
        return self._foo

é equivalente a:

    def foo(self)
        return self._foo
    foo = property(foo)

Portanto, a função  foo()  é substituída pela propriedade   property(foo) . Então, se você usa
 @foo.setter() , o que você está fazendo é chamar o método  property().setter 
Desta  maneira,  podemos  chamar  esses  métodos  sem  os  parênteses,  como  se  fossem  atributos
públicos.  É  uma  forma  mais  elegante  de  encapsular  nossos  atributos.  Vamos  testar  criar  uma  conta  e
depois atribuir um valor negativo ao saldo:

    >>> conta = Conta(1000.0)
    >>> conta.saldo = -300.0
    "saldo não pode ser negativo"

Veja que temos um resultado muito melhor do que usar getters and setters diretamente. Chamamos o
atributo pela suas propriedades, que podem conter validações, e nossos atributos estão sinalizados como
'protegidos' através do '_'.
Mas ainda podemos modificar o saldo e isto deveria ser feito através dos métodos públicos   saca() 
e    deposita()  .  Então,  a  necessidade  de  um    @saldo.setter    é  questionável.  Devemos  apenas
manipular  o  saldo  através  dos  métodos    saca()    e    deposita()  ,  não  precisamos  da  property
 saldo.setter . Isso é uma decisão de negócio específico. O programador deve ficar alerta quanto as
propriedades setters de seus atributos, nem sempre elas são necessárias.
É  uma  má  prática  criar  uma  classe  e,  logo  em  seguida,  criar  as  propriedades  para  todos  seus
atributos.  Você  só  deve  criar  properties  se  tiver  real  necessidade.  Repare  que  nesse  exemplo,  a
propriedade  setter  do  saldo  não  deveria  ter  sido  criada  já  que  queremos  que  todos  usem  os  métodos
 deposita()  e  saca() .

### 8.2 ATRIBUTOS DE CLASSE
Nosso  banco  também  quer  controlar  a  quantidade  de  contas  existentes  no  sistema.  Como
poderíamos fazer isso? Bom, a cada instância criada deveríamos incrementar esse total:

    total_contas = 0
    conta = Conta(300.0)
    total_contas = total_contas + 1
    conta2 = Conta(100.0)
    total_contas = total_contas + 1

Aqui Volta o problema de repetir um mesmo código para toda aplicação, além de ter que lembrar de
incrementar a variável   total_contas  toda vez após instanciar uma   Conta .  Como   total_contas 
tem vínculo com a classe   Conta , ele deve ser um atributo controlado pela classe que deve incrementálo toda vez que instanciamos um objeto, ou seja, quando chamamos o método  `__init__()` :

    class Conta:
        def __init__(self, saldo):
            self._saldo = saldo
            self._total_contas = self._total_contas + 1

Mas onde inicializamos a variável   _total_contas ? Não faz sentido recebermos por parâmetro no
 `__init__()`  já que é a classe que deve controlar esse número e não o objeto. Seria interessante que essa
variável  fosse  própria  da  classe,  fosse  única  e  compartilhada  por  todos  os  objetos  dessa  classe.  Dessa
maneira,  quando  mudasse  através  de  um  objeto,  o  outro  enxergaria  o  mesmo  valor.  Para  fazer  isso,
vamos inicializar a variável na classe, portanto, fora do método  `__init__()` :

    class Conta:
        total_contas = 0
        def __init__(self, saldo):
            self._saldo = saldo
            self.total_contas += 1

Veja que   saldo  é um atributo de instância e   total_contas  um atributo de classe. Vamos fazer
um teste para ver se nosso  total_contas  funciona como esperado:

    >>> c1 = Conta(100.0)
    >>> c1.total_contas
    1
    >>> c2 = Conta(200.0)
    >>> c2.total_contas
    1

Criamos duas instâncias e mesmo assim o   total_contas  não mudou. Isso acontece por conta do
 self.total_contas  +=  1 .   self.total_contas   é  diferente  de   total_contas   da  classe.  Como
 total_contas  é uma variável da classe, devemos chamá-la pela classe:

    class Conta:
        total_contas = 0
        def __init__(self, saldo):
            self._saldo = saldo
            Conta.total_contas += 1

E testamos:

    >>> c1 = Conta(100.0)
    >>> c1.total_contas
    1
    >>> c2 = Conta(200.0)
    >>> c2.total_contas
    2

Agora obtemos o resultado esperado. Também é possível acessar este atributo direto da classe:

    >>> Conta.total_contas
    2

Mas  não  queremos  que  ninguém  venha  a  acessar  nosso  atributo   total_contas   e  modificá-lo.
Portanto vamos torná-lo 'protegido' acrescentando um '_':

    class Conta:
        _total_contas = 0

Dessa maneira avisamos os usuários de nossa classe que esse atributo deve ser considerado 'privado'
e não modificado. Mas como acessá-lo então? Veja que agora, ao acessar pela classe obtemos um erro:

    >>> Conta.total_contas
    Traceback (most recent call last):
    File <stdin>, line 23, in <module>
        Conta.total_contas
    AttributeError: 'Conta' object has no attribute 'total_contas'

Precisamos criar um método para acessar o atributo. Vamos criar o  get_total_contas :

    class Conta:
        _total_contas = 0
        # __init__ e outros métodos
        def get_total_contas(self):
            return Conta._total_contas

Funciona  quando  chamamos  este  método  por  um  instância,  mas  quando  fazemos
 Conta.get_total_contas()  o interpretador reclama pois não passamos a instância:

    >>> c1 = Conta(100.0)
    >>> c1.get_total_contas()
    1
    >>> c2 = Conta(200.0)
    >>> c2.get_total_contas()
    2
    >>> Conta.get_total_contas()
    Traceback (most recent call last):
    File <stdin>, line 17, in <module>
        Conta.get_total_contas()
    TypeError: get_total_contas() missing 1 required positional argument: 'self'

Veja  que  o  erro  avisa  que  falta  passar  o  argumento   self .  Não  podemos  chamá-lo  pois  não  está
vinculado  a  qualquer  instância  de   Conta .  E  um  método  quer  uma  instância  como  seu  primeiro
argumento:

    >>> c1 = Conta(100.0)
    >>> c2 = Conta(200.0)
    >>> Conta.get_total_contas(c1)
    2

Passamos a instância  c1  de  Conta  e funcionou. Mas essa não é a melhor maneira de se chamar um
método. A chamada não é clara e leva um tempo para ler e entender o que a terceira linha desse código
realmente faz. Vamos então deixar de passar o 'self' como argumento de  get_total_contas :

    def get_total_contas():
        return Conta._total_contas

Mas  dessa  maneira  não  conseguimos  acessar  o  método  já  que  todo  método  exige  o  argumento
 self :

    >>> c1 = Conta(100.0)
    >>> c1.get_total_contas()
    Traceback (most recent call last):
    File <stdin> in <module>
        c1.get_total_contas()
    TypeError: get_total_contas() takes 0 positional arguments but 1 was given

E  agora,  o  que  fazer?  Queremos  um  método  que  seja  chamado  via  classe  e  via  instância  sem  a
necessidade de passar a referência deste objeto. O Python resolve isso usando métodos estáticos.
Métodos  estáticos  não  precisam  de  uma  referência,  não  recebem  um  primeiro  argumento  especial
(self). É como uma função simples que, por acaso, reside no corpo de uma classe em vez de ser definida
no nível do módulo.
Para que um método seja considerado estático basta adicionarmos um decorador, como fizemos com
as propriedades no capítulo anterior. O decorador se chama @staticmethod:

    @staticmethod
    def get_total_contas():
        return Conta._total_contas

Testando, vemos que funciona tanto chamado por um instância quanto pela classe:

    >>> c1 = Conta(100.0)
    >>> c1.get_total_contas()
    1
    >>> c2 = Conta(200.0)
    >>> c2.get_total_contas()
    2
    >>> Conta.get_total_contas()
    2

### 8.3 MÉTODOS DE CLASSE
Métodos estáticos não devem ser confundidos com métodos de classe. Como os métodos estáticos,
métodos de classe não são ligados às instâncias, mas sim a classe. O primeiro parâmetro de um método
de classe é uma referência para a classe, isto é, um objeto do tipo class que por convenção nomeamos
como  'cls'.  Eles  podem  ser  chamados  via  instância  ou  pela  classe  e  utilizam  um  outro  decorar,  o

    @classmethod:
    class Conta:
        _total_contas = 0
        def __init__(self):
            type(self)._total_contas += 1
        @classmethod
        def get_total_contas(cls):
            return cls._total_contas

E podemos testar:

    >>> c1 = Conta(100.0)
    >>> c1.get_total_contas()
    1
    >>> c2 = Conta(200.0)
    >>> c2.get_total_contas()
    2
    >>> Conta.get_total_contas()
    2

No início pode parecer confuso qual usar:   @staticmethod  ou   @classmethod ? Isso não é trivial.
Métodos  de  classe  servem  para  definir  um  método  que  opera  na  classe,  e  não  em  instâncias.  Já  os
métodos estáticos utilizamos quando não precisamos receber a referência de um objeto especial (seja da
classe ou de uma instância) e funciona como uma função comum, sem relação.
Isso  ficará  mais  claro  quando  avançarmos  no  aprendizado.  No  próximo  capítulo  discutiremos
Herança,  um  conceito  fundamental  em  Orientação  a  Objetos.  Veremos  que  classes  podem  ter  filhas  e
aproveitar o código das classes mães. Um método de classe pode mudar a implementação, ou seja, pode
ser  reescrito  pela  classe  filha.  Já  os  métodos  estáticos  não  podem  ser  reescritos  pelas  filhas,  já  que  são
imutáveis e não dependem de um referência especial.
@CLASSMETHOD X @STATICMETHOD
Alguns  programadores  não  veem  muito  sentido  em  usar  métodos  estáticos,  já  que  se  você
escrever uma função que não vai interagir com a classe, basta defini-la no módulo. Outros já contra
argumentam  em  outra  via,  considerando  herança  de  classes  que  veremos  em  outro  capítulo.
Indicamos  a  leitura  do  artigo  'The  Definitive  Guide  on  How  to  Use  Static,  Class  and  Abstract
Methods  in  Python'  de  Julien  Danjou  que  pode  ser 
acessado 
pelo 
link: https://julien.danjou.info/guide-python-static-class-abstract-methods/

### 8.4 PARA SABER MAIS - SLOTS
Aprendemos  sobre  encapsulamento  e  vimos  que  é  uma  boa  prática  proteger  nossos  atributos
incluindo  o  prefixo  underscore  em  seus  nomes,  seguindo  a  convenção  utilizada  pelos  programadores.
Além  disso,  utilizamos  properties  para  acessar  e  modificar  nossos  atributos.  Mas  como  Python  é  uma
linguagem dinâmica, nada impede que usuários de nossa classe   Conta   criem  atributos  em  tempo  de
execução, fazendo, por exemplo:

    >>> conta.nome = "minha conta"

Esse  código  não  acusa  erro  e  nossa  conta  fica  aberta  a  modificações  ferindo  a  segurança  da  classe.
Para  evitar  isso  podemos  utilizar  uma  variável  embutida  no  Python  chamada   `__slots__`   que  pode
guardar uma lista de atributos da classe definidos por nós:

    class Conta:
        __slots__ = ['_numero', '_titular', '_saldo', '_limite']
        def __init__(self, numero, titular, saldo, limite=1000.0):
            # inicialização dos atributos
        # código omitido

Agora, quando tentamos adicionar um atributo na classe recebemos um erro:

    >>> conta.nome = "minha_conta"
    Traceback (most recent call last):
        File <stdin>, line 1, in <module>
    AttributeError: 'Conta' object has no attribute '__dict__'
    class Conta:
        __slots__ = ['_numero', '_titular', '_saldo', '_limite']
        def __init__(self, numero, titular, saldo, limite=1000.0):
            self.numero = numero
            self.titular = titular
            self.saldo = saldo
            self.limite = limite
        # restante do código
    conta.nome = "minha_conta"

Repare que o erro acusa que a classe  Conta  não possui o atributo  `__dict__` . Ao atribuir um valor
para  `__slots__` , o interpretador do Python vai entender que queremos excluir o   `__dict__`  da classe
 Conta  não sendo possível criar atributos, ou seja, impossibilitando adicionar atributos ao dicionário da
classe que é responsável por armazenar atributos de instância. Portanto, tentar chamar   vars(conta) 
também vai gerar um erro:

    >>> vars(conta)
    Traceback (most recent call last):
    File <stdin>, line 1, in <module>
    TypeError: vars() argument must have __dict__ attribute

Embora   `__slots__`   seja  muito  utilizado  para  não  permitir  que  usuários  de  nossas  classes  criem
outros atributos, essa não é sua principal função nem o motivo de sua existência. O que acontece é que o
 `__dict__`    desperdiça  muita  memória.  Imagine  um  sistema  grande,  com  milhões  de  instâncias  de
 Conta  - teríamos, consequentemente, milhões de dicionários de classe armazenando seus atributos de
instância. O Python não pode simplesmente alocar uma quantidade estática de memória na criação de
objetos para armazenar todos os atributos. Por isso, consome muita memória RAM se você criar muitos
objetos.
Para  contornar  este  problema  é  que  se  usa  o   `__slots__`   e  este  é  seu  principal  propósito.  O
 `__slots__`  avisa o Python para não usar um dicionário e apenas alocar espaço para um conjunto fixo
de atributos.
Programadores viram uma redução de quase 40 a 50% no uso de RAM usando essa técnica.

### 8.5 EXERCÍCIOS:
1.  Adicione o modificador de visibilidade privado (dois underscores:  __ ) para cada atributo e método
da sua classe  Conta . Tente criar uma  Conta  e modificar ou ler um de seus atributos "privados". O
que acontece?
2.  Sabendo  que  no  Python  não  existem  atributos  privados,  como  podemos  modificar  e  ler  esses
atributos? É uma boa prática fazer isso?
3.  Modifique o acesso para 'protegido' seguindo a convenção do Python e modifique o prefixo  __  por
apenas  um  underscore   _ .  Crie  métodos  de  acesso  em  sua  classe   Conta   através  do  decorator
 @property .
4.  Crie novamente uma conta e acesse e modifique seus atributos. O que mudou?
5.  Modifique  sua  classe   Conta   de  modo  que  não  seja  permitido  criar  outros  atributos  além  dos
definidos anteriormente utilizando  `__slots__` .
6.  (Opcional) Adicione um atributo  identificador  na classe  Conta . Esse identificador deve ter um
valor único para cada instância do tipo  Conta . A primeira Conta instanciada tem identificador 1, a
segunda  2,  e  assim  por  diante.  Você  deve  utilizar  os  recursos  aprendidos  aqui  para  resolver  esse
problema.

# CAPÍTULO 9
## HERANÇA E POLIMORFISMO
### 10.1 REPETINDO CÓDIGO?
Como  toda  empresa,  nosso  banco  possui funcionários.  Um  funcionário  tem  um  nome,  um  cpf  e  um salário. Vamos modelar a classe `Funcionario`:

In [0]:
class Funcionario:
    def __init__(self, nome, cpf, salario):
        self._nome = nome
        self._cpf = cpf
        self._salario = salario
    # outros métodos e propriedades

Além de um funcionário comum, há também outros cargos, como os gerentes. Os gerentes guardam a mesma  informação  que  um  funcionário  comum,  mas possuem  outras  informações,  além  de  ter funcionalidades um pouco diferentes. Um gerente no nosso banco possui também uma senha numérica que permite o acesso ao sistema interno do banco, além do número de funcionários que ele gerencia:

In [0]:
class Gerente:
    def __init__(self, nome, cpf, salario, senha, qtd_gerenciados):
        self._nome = nome
        self._cpf = cpf
        self._salario = salario
        self._senha = senha
        self._qtd_gerenciados = qtd_gerenciados
    def autentica(self, senha):
        if self._senha == senha
            print("acesso permitido")
            return True
        else:
           print("acesso negado")
           return False
    # outros métodos (comuns a um Funcionario)

Se  tivéssemos  um  outro  tipo  de  funcionário  que  tem  características  diferentes  do  funcionário
comum, precisaríamos criar uma outra classe e copiar o código novamente.

Além  disso,  se  um  dia  precisarmos  adicionar  uma  nova  informação  para  todos  os  funcionários,
precisaremos passar por todas as classes de funcionário e adicionar esse atributo. O problema acontece novamente por não centralizarmos as informações principais do funcionário em um único lugar!

Existe um jeito de relacionarmos uma classe de tal maneira que uma delas herda tudo que o outra tem.Isto  é  uma  relação  de  herança,  uma relação  entre  classe  'mãe'  e  classe  'filha'.

No  nosso  caso, gostaríamos de fazer com que   `Gerente`  tivesse tudo que um   `Funcionario`  tem, gostaríamos que ela fosse uma extensão de  `Funcionario`. Fazemos isso acrescentando a classe mãe entre parenteses junto a
classe filha:

In [0]:
class Gerente(Funcionario):
    def __init__(self, senha, qtd_funcionarios):
        self._senha = senha
        self._qtd_funcionarios = qtd_funcionarios
    def autentica(self, senha):
        if self._senha == senha:
            print("acesso permitido")
            return True
        else:
            print("acesso negado")              
            return False

Todo momento que criarmos um objeto do tipo `Gerente`  queremos que este objeto também herde
os atributos definidos na classe  `Funcionario` , pois um `Gerente` é um `Funcionário`.

Como  a  classe   `Gerente`   já  possui  um  método   `__init__()`   com  outros  atributos,  o  método  da classe   `Funcionario`   é  sobrescrito  pelo   `Gerente`.  Se  queremos  incluir  os  mesmos  atributos  de instância  de    `Funcionario`    em  um    `Gerente`    devemos  chamar  o  método    `__init__()` de `Funcionario`  dentro do método  `__init__()`  de `Gerente`:

In [0]:
class Gerente(Funcionario):
    def __init__(self, senha, qtd_funcionarios):
        Funcionario.__init__(nome, cpf, salario)
        self._senha = senha
        self._qtd_funcionarios = qtd_funcionarios
    def autentica(self, senha):
        if self._senha == senha:
            print("acesso permitido")
            return True
        else:
            print("acesso negado")              
            return False

Dizemos que a classe   `Gerente`  herda todos os atributos e métodos da classe mãe, no nosso caso, a `Funcionario`. Como Python tem tipagem dinâmica,  precisamos garantir isso através do construtor da
classe. Além de  `senha`  e  `qtd_funcionarios`  passamos também os atributos  `nome` ,  `cpf`  e   `salario` que todo funcionário tem:

In [0]:
class Gerente(Funcionario):
    def __init__(self, nome, cpf, salario, senha, qtd_funcionarios):
        self._senha = senha
        self._qtd_funcionarios = qtd_funcionarios

Como  estes  são  atributos  de  um   `Funcionario`   e  não  queremos  repetir  o  código  do  método
 `__init__()`  de  `Funcionario`  dentro da classe  `Gerente` , podemos chamar este método da classe mãe
como fizemos no exemplo acima ou podemos utilizar um método do Python chamado `super()`:

In [0]:
class Gerente(Funcionario):
    def __init__(self, nome, cpf, salario, senha, qtd_funcionarios):
        super().__init__(nome, cpf, salario)
        self._senha = senha
        self._qtd_funcionarios = qtd_funcionarios

Para  ser  mais  preciso,  ela  também  herda  os  atributos  e  métodos  'privados'  de   `Funcionario`.

O `super()` é  usado  para  fazer  referência  a  superclasse,  a  classe  mãe  -  no  nosso  exemplo  a  classe `Funcionario`.

### PARA SABER MAIS: SUPER E SUB CLASSE
A nomenclatura mais encontrada é que `Funcionario` é a superclasse de `Gerente`, e `Gerente` é a subclasse  de  `Funcionario`.  Dizemos  também  que  todo  `Gerente`  é um  `Funcionario`.  Outra  forma  é
dizer que `Funcionario` é a classe mãe de `Gerente` e `Gerente` é a classe filha de Funcionario.

Da  mesma  maneira,  podemos  ter  uma  classe    `Diretor`    que  estenda    `Gerente`    e  a  classe
 `Presidente`   pode  estender  diretamente  de  `Funcionario` .  Fique  claro  que  essa  é  uma relação  de negócio.  Se   `Diretor`   vai  estender  de   `Gerente`   ou  não,  vai  depender,  para  você,   `Diretor`   é  um `Gerente`?

### 9.2 REESCRITA DE MÉTODOS
Todo  fim  de  ano,  os  funcionários  do  nosso  banco  recebem  uma  bonificação.  Os  funcionários comuns recebem 10% do valor do salário e os gerentes, 15%.

Vamos ver como fica a classe  `Funcionario`:

In [0]:
class Funcionario:
    def __init__(self, nome, cpf, salario):
        self._nome = nome
        self._cpf = cpf
        self._salario = salario
    # outros métodos e propriedades
    def get_bonificacao(self):
        return self._salario * 0.10

Se deixarmos a classe  Gerente  como ela está, ela vai herdar o método  `get_bonificacao()`: 

In [0]:
gerente = Gerente('José', '222222222-22', 5000.0, '1234', 0)
print(gerente.get_bonificacao())

O resultado aqui será `500`. Não queremos essa resposta, pois o gerente deveria ter `750` de bônus nesse caso. Para consertar isso, uma das opções seria criar um novo método na classe  `Gerente` , chamado, por exemplo,  `get_bonificacao_do_gerente()`. O problema é que teríamos dois métodos em `Gerente`,confundindo  bastante  quem  for  usar  essa  classe,  além  de  que  cada  um  gerenciaria  uma  resposta diferente.

No  Python,  quando  herdamos  um  método,  podemos  alterar  seu  comportamento.  Podemos reescrever (sobrescrever, override) este método, assim como fizemos com o  `__init__` :

In [0]:
class Gerente(Funcionario):
    def __init__(self, nome, cpf, salario, senha, qtd_gerenciaveis):
        super().__init__(nome, cpf, salario)
        self._senha = senha
        self._qtd_gerenciaveis = qtd_gerenciaveis
    def get_bonificacao(self):
        return self._salario * 0.15
    # metodos e properties

Agora o método está correto para o  `Gerente` . Refaça o teste e veja que o valor impresso é o correto (`750`):

In [0]:
gerente = Gerente('José', '222222222-22', 5000.0, '1234', 0)
print(gerente.get_bonificacao())

Utilize o método   `vars()`  para acessar os atributos de   Gerente  e ver que a classe herda todos os atributos de  `Funcionario`:

In [0]:
funcionario = Funcionario('João', '111111111-11', 2000.0)
print(vars(funcionario))

In [0]:
gerente = Gerente('José', '222222222-22', 5000.0, '1234', 0)
print(vars(gerente))

### 9.3 INVOCANDO O MÉTODO REESCRITO
Depois  de  reescrito,  não  podemos  mais  chamar  o  método  antigo  que  fora  herdado  da  classe  mãe, realmente  alteramos  o  seu  comportamento.  Mas  podemos  invocá-lo  no  caso  de  estarmos  dentro  da classe.

Imagine  que  para  calcular  a  bonificação  de  um   `Gerente`   devemos  fazer  igual  ao  cálculo  de  um Funcionario  adicionando `1000.0` reais. 

Poderíamos fazer assim:

In [0]:
class Gerente(Funcionario):
    def __init__(self, senha, qtd_gerenciaveis):
        self._senha = senha
        self._qtd_gerenciaveis = qtd_gerenciaveis
    def get_bonificacao():
        return self._salario * 0.10 + 1000.0
    # métodos e propriedades

Aqui  teríamos  um  problema:  o  dia  que  o    `get_bonificacao()`    do    `Funcionario`    mudar,
precisaremos mudar o  método do   `Gerente`   para  acompanhar  a  nova  bonificação.  Para  evitar  isso,  o `get_bonificacao()`  do  Gerente  pode chamar o do  `Funcionario`  utilizando o método `super()`.

In [0]:
class Gerente(Funcionario):
    def __init__(self, senha, qtd_gerenciaveis):
        self._senha = senha
        self._qtd_gerenciaveis = qtd_gerenciaveis
    def get_bonificacao():
        return super().get_bonificacao() + 1000
    # métodos e propriedades

Essa  invocação  vai  procurar  o  método  com  o  nome   `get_bonificacao()`  de  uma  superclasse  de
 `Gerente`. No caso, ele logo vai encontrar esse método em  `Funcionario`.

Essa é uma prática comum, pois em muitos casos o método reescrito geralmente faz algo a mais que
o método da classe mãe. Chamar ou não o método de cima é uma decisão e depende do seu problema.

Algumas vezes não faz sentido invocar o método que reescrevemos.

Para  escrever  uma  classe  utilizando  o  Python  2  é  preciso  acrescentar  a  palavra  `object` quando definimos uma classe:

In [0]:
class MinhaClasse(object):
    pass

Isso  acontece  porque  toda  classe  é  filha  de  `object`  -  que  é  chamada  a  mãe  de  todas  as  classes.  No Pyhton, toda classe herda de `object`. No Python 3 não precisamos acrescentar o `object` mas não quer dizer que esta classe e a herança não existam, apenas que essa herança é implícita. Quando criamos uma classe vazia e utilizamos o método  `dir()`  para checar a lista de seus atributos, reparamos que ela não é vazia:

In [0]:
class MinhaClasse():
    pass
if __name__ == '__main__':
    mc = MinhaClasse()
    print(dir(mc))

Todos estes atributos são herdados da classe `object` e podemos reescrever qualquer um deles na nossa
subclasse. Todos eles são os conhecidos métodos 'mágicos' (começam e iniciam com dois underscores, e
por este motivo, também chamados de dunders).

Vimos o comportamento do   `__init__()`,  `__new__()`  e do   `__dict__`. Outros métodos mágicos
famosos são  `__str__()`  e   `__repr__()`  - métodos que retornam a representação do objeto como uma
`string`. Quando chamamos  `print(mc)`  temos a saída

    <__main__.MinhaClasse object at 0x7f11c1f59a58>

Esse  é  o  modelo  padrão  de  impressão  de  um  objeto,  implementado  na  classe   `object`.  A  função `print()` na  verdade  usa  a   `string`    definida  pelo  método   `__str__()` de  uma  classe.  Vamos reescrever este método:

In [0]:
class MinhaClasse:
    def __str__(self):
        return '< Instância de {}; endereço:{}>'.format(self.__class__.__name__, id(self))

Agora, quando executamos  `print(mc)` , a saída é:

In [0]:
print(mc)

O Python sempre chama o método `__str__()`  quando utiliza a função   `print()`  em um objeto.

Novamente, estamos utilizando reescrita de métodos.
O método   `__repr__()`  também retorna uma   `string`  e podemos utilizar a função   `repr()`  para
checar seu retorno:

In [0]:
print(repr(mc))

Que vai gerar a mesma saída padrão do  `__str__()`:

In [0]:
print(mc.__str())

Mas  diferente  do   `__str__()`,  não  é  comum  sobrescrever  este  método.  Ele  é  sobrescrito  quando precisamos utilizá-lo junto com a função  `eval()`  do Python. A função  `eval()`  recebe uma   `string` e tenta executar essa  `string`  como um comando do Python, veja um exemplo de uso:

In [0]:
x = 1

In [0]:
eval("x+1")

Vamos a um exemplo utilizando classes:

In [0]:
class Ponto:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "({}, {})".format(self.x, self.y)
    def __repr__(self):
        return "Ponto({}, {})".format(self.x + 1, self.y + 1)

In [0]:
if __name__ == '__main__':
    p1 = Ponto(1, 2)
    p2 = eval(repr(p1))
    print(p1)
    print(p2)

Repare que utilizamos a função `repr()`  passando uma instância de   `Ponto`. O Python vai chamar então o método  `__repr__()`  da classe  `Ponto`, que retorna a  `string`  `"Ponto(2, 3)"` já que   `p1.x = 1` 
e  `p1.y = 2`. Ao passá-la de argumento para a função  `eval()`, teremos:  `p2 = eval('Ponto(2, 3))`.

Como a função  `eval()`  vai tentar executar essa  string  como um comando Python válido, ele vai ter
sucesso e portanto  `p2`  será uma nova instância da classe  `Ponto`  com  `p2.x = 2`  e  `p2.y = 3`.

Para concluir, é importante entender que tanto   `__str__()`  quanto   `__repr__()`   retornam  uma
 string  que representa o objeto mas com propósitos diferentes. O método  `__str__()`  é utilizado para
apresentar mensagens para os usuários da classe, de maneira mais amigável. Já o método  `__repr__()`  é
usado para representar o objeto de maneira técnica, inclusive podendo utilizá-lo como comando válido do Python como vimos no exemplo da classe  `Ponto`.

### 9.4 PARA SABER MAIS - MÉTODOS MÁGICOS
Os  métodos  mágicos  são  úteis  pois  permitem  que  os  objetos  de  nossas  classes  possuam  uma
interface  de  acesso  semelhante  aos  objetos  embutidos  do  Python.  O  método   `__add__()`,  por
exemplo,  serve  para  executar  a  adição  de  dois  objetos  e  é  chamada  sempre  quando  fazemos  a
operação de adição (`obj + obj`) utilizando o operador `+`. Por exemplo, quando fazemos  `1 + 1`  no
Python, o que o interpretador faz é chamar o método  `__add__()`  da classe  `int`. Vimos que uma
 `list`  também implementa o método  `__add__()`  já que a operação de adição é definida para esta classe:

In [0]:
lista = [1, 2, 3]

In [0]:
lista + [4, 5]

In [0]:
print(lista)

O  mesmo  ocorre  para  as  operações  de multiplicação,  divisão,  módulo  e  potência  que  são definidas  pelos  métodos  mágicos `__mul__()`, `__div__()`, `__mod__()` e `__pow__()`,
respectivamente.

Podemos  definir  cada  uma  dessas  operações  em  nossas  classes  sobrescrevendo  tais  métodos
mágicos.  Além  desses  o  Python  possui  muitos  outros  que  você  pode  acessar  aqui:

https://docs.python.org/3/reference/datamodel.html#Basic_customization

### 9.5 POLIMORFISMO
O que guarda uma variável do tipo  `Funcionario`? Uma referência para um  `Funcionario` , nunca o objeto em si.

Na herança, vimos que todo   `Gerente`  é um `Funcionario`,  pois  é  uma  extensão  deste.

Podemos nos  referir  a  um    `Gerente`    como  sendo  um    `Funcionario`.  Se  alguém  precisa  falar  com  um `Funcionario`  do banco, pode falar com um   `Gerente`! Porque? Pois   `Gerente`  é um   `Funcionario`.

Essa é a semântica da herança.

Polimorfismo  é  a  capacidade  de  um  objeto  poder  ser  referenciado  de  várias  formas.  (cuidado, polimorfismo não quer dizer que o objeto fica se transformando, muito pelo contrário, um objeto nasce de um tipo e morre daquele tipo, o que pode mudar é a maneira como nos referimos a ele).

A  situação  que  costuma  aparecer  é  a  que  temos  um  método  que  recebe  um  argumento  do  tipo `Funcionario`:

In [0]:
class ControleDeBonificacoes:
    def __init__(self, total_bonificacoes=0):
        self._total_bonificacoes = total_bonificacoes
    def registra(self, funcionario):
        self._total_bonificacoes += funcionario.get_bonificacao()
    @property
    def total_bonificacoes(self):
        return self._total_bonificacoes

E podemos fazer:

In [0]:
if __name__ == '__main__':
    funcionario = Funcionario('João', '111111111-11', 2000.0)
    print("bonificacao funcionario: {}".format(funcionario.get_bonificacao()))
    gerente = Gerente("José", "222222222-22", 5000.0 '1234', 0)
    print("bonificacao gerente: {}".format(gerente.get_bonificacao()))
    controle = ControleDeBonificacoes()
    controle.registra(funcionario)
    controle.registra(gerente)
    print("total: {}".format(controle.total_bonificacoes))

Repare  que  conseguimos  passar  um   `Gerente` para  um  método  que  "recebe"  um   `Funcionario` como  argumento.  Pense  como  numa  porta  na  agência  bancária  com  o  seguinte  aviso:  
"Permitida  a entrada  apenas  de  Funcionários".  Um  gerente  pode  passar  nessa  porta?  Sim,  pois   `Gerente`   é  um `Funcionario`.

Qual  será  o  valor  resultante?  Não  importa  que  dentro  do  método    `registra()`    do `ControleDeBonificacoes`  receba   `Funcionario`. Quando ele receber um objeto que realmente é um
 `Gerente`, o seu método reescrito será invocado. Reafirmando: não importa como nos referenciamos a
um objeto, o método que será invocado é sempre o que é dele.

No  dia  em  que  criarmos  uma  classe   `Secretaria`,  por  exemplo,  que  é  filha  de   `Funcionario`, precisaremos mudar a classe   `ControleDeBonificacoes`? Não. Basta a classe   `Secretaria`  reescrever os métodos que lhe parecerem necessários. É exatamente esse o poder do polimorfismo, juntamente com a reescrita de método: diminuir o acoplamento entre as classes, para evitar que novos códigos resultem em modificações em inúmeros lugares.

Repare que quem criou   `ControleDeBonificacoes`   pode  nunca  ter  imaginado  a  criação  da  classe
 `Secretaria`   ou   `Engenheiro`. Contudo,  não  será  necessário  reimplementar  esse  controle  em  cada nova classe: reaproveitamos aquele código.
Pensar  desta  maneira  em  linguagens  com  tipagem  estática  é  o  mais  correto  já  que  as  variáveis  são tipadas  e  garantem,  através  do  compilador,  que  o  método  só  funcionará  se  receber  um  tipo `Funcionario`.  Mas  não  é  o  que  acontece  em  linguagens  de  tipagem  dinâmica  como  Python.  Vamos supor que temos uma classe para representar os clientes do banco:

In [0]:
class Cliente:
    def __init__(self, nome, cpf, senha):
        self._nome = nome
        self._cpf = cpf
        self._senha = senha
    # métodos e propriedades

Nada  impede  de  registrarmos  um   `Cliente`   em   `ControleDeBonificacoes`.  Vamos  ver  o  que
acontece:

In [0]:
cliente = ('Maria', '333333333-33', '1234')
controle = ControleBonificacoes()
controle.registra(cliente)

Veja que lança um  `AttibuteError`  com a mensagem dizendo que   `Cliente`  não possui o atributo
 `get_bonificacao`.  Portanto,  aqui  não  importa  se  o  objeto  recebido  no  método   `registra()`  é  um `Funcionario`, mas se ele possui o método  `get_bonificacao()`.

O método   `registra()`   utiliza  um  método  da  classe   `Funcionario`   e,  portanto,  funcionará  com qualquer  instância  de  uma  subclasse  de    `Funcionario`    ou  qualquer  instância  de  uma  classe  que implemente o método  `get_bonificacao()`.

Podemos  evitar  este  erro  verificando  se  o  objeto  passado  possui  ou  não  um  atributo
 `get_bonificacao()`  através da função  `hasattr()` :

In [0]:
class ControleDeBonificacoes:
    def __init__(self, total_bonificacoes=0):
        self._total_bonificacoes = total_bonificacoes
    def registra(self, obj):
        if(hasattr(obj, 'get_bonificacao'))
            self._total_bonificacoes += obj.get_bonificacao()
        else
            print('instância de {} não implementa o método get_bonificacao()'.format(self.__class__._
_name__))    
    # demais métodos

A  função   `hasattr()`   recebe  dois  parâmetros,  o  objeto  e  o  atributo  (na  forma  de   `string` )  -  everifica se o objeto possui aquele atributo, ou seja, se o atributo está contido no   `__dict__`  do objeto.

Então, fazemos a pergunta:   `get_bonificacao()`  é atributo de   `Funcionario`? Se sim, entra no bloco
 `if`  e podemos chamar o método tranquilamente, evitando erros.

Agora, se tentarmos chamar o método  `registra()`  passando um  Cliente  recebemos a saída:

    'Cliente' object has no attribute 'get_bonificacao

Portanto, o tipo passado para o método   `registra()`  não importa aqui e sim se o objeto passado implementa  ou  não  o  método `get_bonificacao()`.

Ou  seja,  basta  que  o  objeto  atenda  a  um
determinado protocolo.

Existe uma função no Python que funciona de forma semelhante mas considera o tipo da instância, é
a função `isinstance()`. Ao invés de passar uma instância, passamos a classe no segundo parâmetro.

In [0]:
class ControleDeBonificacoes:
    def __init__(self, total_bonificacoes=0):
        self.__total_bonificacoes = total_bonificacoes
    def registra(self, obj):
        if(isinstance(obj, Funcionario)):
            self.__total_bonificacoes += obj.get_bonificacao()
        else:
            print('instância de {} não implementa o método get_bonificacao()'
                .format(self.__class__.__name__))

Mas essa não é a maneira Pythônica. Você deve escrever o código esperando somente uma interface
do objeto, não o tipo dele. A interface é o conjunto de métodos públicos de uma classe. No caso da nossa
classe   `ControleDeBonificacoes`,  o  método   `registra()`   espera  um  objeto  que  possua  o  método `get_bonificacao()`  e não um objeto do tipo  `Funcionario`.

### 9.6 DUCK TYPING
Uma característica de linguagens dinâmicas como Python é a chamada Duck Typing, a tipagem de pato. É uma característica de um sistema de tipos em que a semântica de uma classe é determinada pela sua capacidade  de  responder  a  alguma  mensagem,  ou  seja,  responder  a  determinado  atributo  (ou
método). O exemplo canônico (e a razão do nome) é o teste do pato: se ele se parece com um pato, nada
como um pato e grasna como um pato, então provavelmente é um pato.

Veja o exemplo abaixo:

In [0]:
class Pato:
    def grasna(self):
        print('quack!')
class Ganso:
    def grasna(self):
        print('quack!')

pato = Pato()
ganso = Ganso()


In [0]:
pato.grasna()

quack!


In [0]:
ganso.grasna()

quack!


Você deve escrever o código esperando somente uma interface do objeto, não um tipo de objeto. No
caso da nossa classe   `ControleDeBonificacoes`, o método   `registra()`  espera um objeto que possua
o método  `get_bonificacao()`  e não apenas um funcionário.

O Duck Typing é um estilo de programação que não procura o tipo do objeto para determinar se ele
tem  a  interface  correta.  Ao  invés  disso,  o  método  ou  atributo  é  simplesmente  chamado  ou  usado  ('se parece como um pato e grasna como um pato, então deve ser um pato'). Duck Typing evita testes usando as funções   `type()`, `isinstance()`  e até mesmo a `hasattr()` - ao invés disso, deixa o erro estourar na frente do programador.

A maneira Pythônica para garantir a consistência do sistema não é verificar os tipos e atributos de
um objeto, mas pressupor a existência do atributo no objeto e tratar uma exceção, caso ocorra, através
do comando  `try/except` :

In [0]:
try:
    self._total_bonificacoes += obj.get_bonificacao()
except AttributeError as e:
    print(e)

NameError: ignored

Estamos pedindo ao interpretador para tentar executar a linha de código dentro do comando `try` (tentar). Caso ocorra algum erro, ele vai tratar este erro com o comando  `except`  e executar algo, como imprimir o erro (similar ao exemplo). Não se preocupe de entender os detalhes sobre este código e o uso
do  `try/except`  neste momento, teremos um capítulo só para falar deles.

O  que  é  importante  é  que  a  maneira  pythônica  de  se  fazer  é  assumir  a  existência  do atributo  e capturar (tratar) um exceção quando o atributo não pertencer ao objeto e seguir o fluxo do programa.

Por ora, faremos esta checagem utilizando a função  `hasattr()`.

### HERANÇA VERSUS ACOPLAMENTO
Note que o uso de herança aumenta o acoplamento entre as classes, isto é, o quanto uma classe depende de outra. A relação entre classe mãe e filha é muito forte e isso acaba fazendo com que o programador  das  classes  filhas  tenha  que  conhecer  a implementação  da  classe  mãe  e  vice-versa  fica difícil fazer uma mudança pontual no sistema.

Por  exemplo,  imagine  se  tivermos  que  mudar  algo  na  nossa  classe   `Funcionario` ,  mas  não
quiséssemos que todos os funcionários sofressem a mesma mudança. Precisaríamos passar por cada
uma  das  filhas  de   `Funcionario`   verificando  se  ela  se  comporta  como  deveria  ou  se  devemos
sobrescrever o tal método modificado.

Esse é um problema da herança, e não do polimorfismo, que resolveremos mais tarde.

### 9.7 EXERCÍCIO: HERANÇA E POLIMORFISMO
Vamos  ter  mais  de  um  tipo  de  conta  no  nosso  sistema.  Portanto,  além  das  informações  que  já
tínhamos na conta, temos agora o tipo: se queremos uma conta corrente ou uma conta poupança. Além
disso, cada uma deve possuir uma taxa.
1.  Adicione na classe  Conta  um novo método chamado  `atualiza()*`  que atualiza a conta de acordo
com a taxa percentual:
        class Conta:
            #outros métodos
            def atualiza(self, taxa):
                self._saldo += self._saldo * taxa
2.  Crie duas subclasses da classe  `Conta`:  `ContaCorrente`  e   `ContaPoupanca`. Ambas terão o método  `atualiza()` reescrito:  a `ContaCorrente`    deve  atualizar-se  com  o  dobro  da  taxa  e  a `ContaPoupanca`    deve  atualizar-se  com  o  triplo  da  taxa.  Além  disso,  a   `ContaCorrente` deve reescrever  o  método   `deposita()`   afim  de  retirar  uma  taxa  bancária  de  dez  centavos  de  cada depósito.
Crie a classe `ContaCorrente` no arquivo   `conta.py`   e  faça  com  que  ela  seja  subclasse  (filha)  da classe  `Conta`.
        class ContaCorrente(Conta):
            pass
Crie a classe `ContaPoupanca` no arquivo   `conta.py`  e faça com que ela seja subclasse (filha) da
classe  `Conta`:
        class ContaPoupanca(Conta):
            pass
Reescreva o método  `atualiza()`  na classe  `ContaCorrente`, seguindo o enunciado:
        class ContaCorrente(Conta):
            def atualiza(self, taxa):
                self._saldo += self._saldo * taxa * 2
Reescreva o método  `atualiza()`  na classe  `ContaPoupanca`, seguindo o enunciado:
        class ContaPoupanca(Conta):
            def atualiza(self, taxa):
                self._saldo += self._saldo * taxa * 3
Na classe   `ContaCorrente`, reescreva o método   `deposita()`  para descontar a taxa bancária de
dez centavos:
        class Conta Corrente(Conta):
            def atualiza(self, taxa):
                self._saldo += self._saldo * taxa * 2
            def deposita(self, valor):
                self._saldo += valor - 0.10

3.  Agora, teste suas classes no próprio módulo   `conta.py`. Acrescente a condição quando o módulo
for igual a  `__main__`  para executarmos no console. Instancie essas classes, atualize-as e veja o resultado:
        if __name__ == '__main__'
            c = Conta('123-4', 'Joao', 1000.0)
            cc = ContaCorrente('123-5', 'Jose', 1000.0)
            cp = ContaPoupanca('123-6', 'Maria', 1000.0)
            c.atualiza(0.01)
            cc.atualiza(0.01)
            cp.atualiza(0.01)
            print(c.saldo)
            print(cc.saldo)
            print(cp.saldo)

4.  Implemente o método  `__str__()`  na classe  `Conta` . Faça com que ele imprima uma representação
mais amigável de um  `Conta`  contendo todos os seus atributos.
        def __str__(self):
            # sua implementação aqui
Teste chamando o método  `print()`  passando algumas instâncias de  Conta  como argumento.

5.  Vamos criar uma classe que seja responsável por fazer a atualização de todas as contas bancárias e
gerar um relatório com o saldo anterior e saldo novo de cada uma das contas. Na pasta   `src`  crie a
classe  `AtualizadorDeContas`:
        class AtualizadorDeContas:
            def __init__(self, selic, saldo_total=0):
                self._selic = selic
                self._saldo_total = saldo_total
            #propriedades    
            def roda(self, conta):
                #imprime o saldo anterior, atualiza a conta e depois imprime o saldo final
                #soma o saldo final ao atributo saldo_total
Não esqueça de fazer os imports necessário para o código funcionar.

6.  No `main`, vamos criar algumas contas e rodá-las a partir do  `AtualizadorDeContas`:
        if __name__ == '__main__':
            c = Conta('123-4', 'Joao', 1000.0)
            cc = ContaCorrente('123-5', 'José', 1000.0)
            cp = ContaPoupanca('123-6', 'Maria', 1000.0)
            adc = AtualizadorDeContas(0.01)
            adc.roda(c)
            adc.roda(cc)
            adc.roda(cp)
            print('Saldo total: {}'.format(adc.saldo_total))
7.  (opcional)  Se  você  precisasse  criar  uma  classe   `ContaInvestimento`,  e  seu  método   `atualiza()` fosse complicadíssimo, você precisaria alterar a classe  `AtualizadorDeContas`?

8.  (opcional, Trabalhoso) Crie uma classe   `Banco`  que possui uma lista de contas. Repare que em uma
lista de contas você pode colocar tanto `ContaCorrente`  quanto   `ContaPoupanca`. Crie um método `adiciona()`  que adiciona uma conta na lista de contas; um método   `pegaConta()`  que devolve a
conta em determinada posição da lista e outro  `pegaTotalDeContas()`  que retorna o total de contas
na  lista.  Depois  teste  criando  diversas  contas,  insira-as  no   `Banco`   e  depois,  com  um  laço   `for` , percorra  todas  as  contas  do   `Banco`   para  passá-las  como  argumento  para  o  método   `roda()`   do `AtualizadorDeContas`.
9.  (opcional)  Que  maneira  poderíamos  implementar  o  método    `atualiza()`    nas  classes
 `ContaCorrente`  e  `ContaPoupança`  poupando reescrita de código?
10.  (opcional)  E  se  criarmos  uma  classe  que  não  é  filha  de   Conta   e  tentar  passar  uma  instância  no método roda de   `AtualizadorDeContas`? Com o que aprendemos até aqui, como podemos evitar
que erros aconteçam nestes casos?

### 9.8 CLASSES ABSTRATAS
Vamos recordar nossa classe  `Funcionario`:

In [0]:
class Funcionario:
    def __init__(self, nome, cpf, salario=0):
        #inicialização dos atributos
    #propriedades e outros métodos
    def get_bonificacao(self):
        return self._salario * 1.2



Considere agora nosso  ControleDeBonificacao :

In [0]:
class ControleDeBonificacoes:
    def __init__(self, total_bonificacoes=0):
        self.__total_bonificacoes = total_bonificacoes
    def registra(self, obj):
        if(hasattr(obj, 'get_bonificacao')):
            self.__total_bonificacoes += obj.get_bonificacao()
        else:
            print('instância de {} não implementa o método get_bonificacao()'.format(self.__class__._
_name__))  
    #propriedades

Nosso método  `registra()`  recebe um objeto de qualquer tipo mas estamos esperando que seja um
 `Funcionario`   já  que  este  implementa  o  método   `get_bonificacao()`,  isto  é,  podem  ser  objetos  do tipo   `Funcionario`  e qualquer de seus subtipos:   `Gerente`, `Diretor`  e, eventualmente, alguma nova subclasse que venha ser escrita, sem prévio conhecimento do autor da `ControleDeBonificacao`.

Estamos utilizando aqui a classe   `Funcionario`  para o polimorfismo. Se não fosse ela, teríamos um grande  prejuízo:  precisaríamos  criar  um  método   `registra()`   para  receber  cada  um  dos  tipos  de
 `Funcionario`, um para  `Gerente`, um para   `Diretor`, etc. Repare que perder esse poder é muito pior do que a pequena vantagem que a herança traz em herdar código.

Porém, em alguns sistemas, como é o nosso caso, usamos uma classe com apenas esses intuitos: de
economizar  um  pouco  código  e  ganhar polimorfismo  para  criar  métodos  mais  genéricos,  que  se encaixem a diversos objetos.

Faz  sentido  ter  um  objeto  do  tipo    `Funcionario`?  Essa  pergunta  é  bastante  relevante  já  que instanciar  um   `Funcionario`   pode  gerar  um  objeto  que  não  faz  sentido  no  nosso  sistema.  Nossa empresa tem apenas  `Diretores`,  `Gerentes`,  `Secretárias`, etc...   

`Funcionario`  é apenas uma classe que idealiza um tipo, define apenas um rascunho.

Vejamos um outro caso em que não faz sentido ter um objeto de determinado tipo, apesar da classe
existir. Imagine a classe   `Pessoa`  e duas filhas:   `PessoaFisica`  e   `PessoaJuridica`. Quando puxamos um relatório de nossos clientes (uma lista de objetos de tipo  `Pessoa`, por exemplo), queremos que cada um deles seja ou uma  `PessoaFisica`  ou uma  `PessoaJuridica`. A classe   `Pessoa` , nesse caso, estaria sendo  usada  apenas  para  ganhar  o  polimorfismo  e  herdar  algumas  coisas  -  não  faz  sentido  permitir instanciá-la.

Para o nosso sistema, é inadmissível que um objeto seja apenas do tipo   `Funcionario`  (pode existir
um sistema em que faça sentido ter objetos do tipo   `Funcionario`  ou apenas   `Pessoa` , mas, no nosso
caso, não). Para resolver esses problemas, temos as classes abstratas.

Utilizaremos uma módulo do Python chamado `abc` que permite definirmos classes abstratas. Uma classe abstrata deve herdar de `ABC` (Abstract Base Classes). `ABC` é a superclasse para classes abstratas.

Uma classe abstrata não pode ser instanciada e deve conter pelo menos um método abstrato. Vamos ver isso na prática.

Vamos tornar nossa classe  `Funcionario`  abstrata:

In [0]:
import abc
class Funcionario(abc.ABC):
    # métodos e propriedades

Definimos  nossa  classe    `Funcionario` como  abstrata.  Agora  vamos  tornar  nosso  método
 `get_bonificacao()`  abstrato. Um método abstrato pode ter implementação, mas não faz sentido em
nosso  sistema,  portanto  vamos  deixá-lo  sem  implementação.  Para  definir  um  método  abstrato
utilizamos o decorator  `@abstractmethod`:

In [0]:
class Funcionario(abc.ABC):
    @abc.abstractmethod
    def get_bonificacao(self):
        pass

Agora, se tentarmos instanciar um objeto do tipo  `Funcionario`:

In [0]:
f = Funcionario()

Acusa um erro:

    TypeError: Can't instantiate abstract class Funcionario with abstract methods get_bonificacao

Apesar de não conseguir instanciar a classe   `Funcionario`, conseguimos instanciar suas filhas que
são objetos que realmente existem em nosso sistema (objetos concretos):

In [0]:
class Gerente(Funcionario):
    # outros métodos e propriedades
    def get_bonificacao(self):
        return self._salario * 0.15

In [0]:
gerente = Gerente('jose', '222222222-22', 5000.0, '1234', 0)
print(gerente.get_bonificacao())

Vamos criar a classe  `Diretor`  que herda de  `Fucionario`  sem o método  `get_bonificacao()`:

In [0]:
class Diretor(Funcionario):
    def __init__(self, nome, cpf, salario):
        super().__init__(nome, cpf, salario)

In [0]:
diretor = Diretor('joao', '111111111-11', 4000.0)

Não conseguimos instanciar uma subclasse de   `Funcionario`  sem implementar o método abstrato
 `get_bonificacao()`. Agora tornamos o método   `get_bonificacao()`  obrigatório para todo objeto
que  é  subclasse  de    `Funcionario`.  Caso  venhamos  a  criar  outras  classes,  como    `Secretaria` e `Presidente`,  que  sejam  filhas  de    `Funcionario` ,  seremos  obrigados  e  criar  o  método `get_bonificacao()`, caso contrário, o código vai acusar erro quando executado.

### 9.9 EXERCÍCIOS - CLASSES ABSTRATAS
1.  Torne a classe  `Conta`  abstrata.
        import abc
        class Conta(abc.ABC):
        def __init__(self, numero, titular, saldo=0, limite=1000.0):
        self._numero = numero
        self._titular = titular
        self._saldo = saldo
        self._limite = limite
        # outros métodos e propriedades

2.  Torne o método  `atualiza()`  abstrato:
        class Conta(abc.ABC):
        # código omitido
        @abc.abstractmethod
        def atualiza():
            pass

3.  Tente instância uma  `Conta`:
        c = Conta()
O que acontece?

4.  Instancie  uma   `ContaCorrente`   e  uma   `ContaPoupanca`   e  teste  o  código  chamando  o  método `atualiza()`.

        cc = ContaCorrente('123-4', 'João', 1000.0)
        cp = ContaPoupanca('123-5', 'José', 1000.0)
        cc.atualiza(0.01)
        cp.atualiza(0.01)
        print(cc.saldo)
        print(cp.saldo)

5.  Crie uma classe chamada  `ContaInvestimento`:
        class ContaInvestimeto(Conta):
            pass
6.  Instancie uma  `ContaInvestimeto`:
        ci = ContaInvestimento('123-6', 'Maria', 1000.0)

7.  Não  conseguimos  instanciar  uma   `ContaInvestimento`   que  herda   `Conta`   sem  implementar  o método  abstrato    `atualiza()`.  Vamos  criar  uma  implementação  dentro  da  classe `ContaInvestimento`:
        def atualiza(self, taxa):
            self._saldo += self._saldo * taxa * 5

8.  Agora teste instanciando uma  `ContaInvestimento`  e chame o método  `atualiza()`:
        ci = ContaInvestimento('123-6', 'Maria', 1000)
        ci.deposita(1000.0)
        ci.atualiza(0.01)
        print(ci.saldo)

9.  (opcional)  Crie  um  atributo    `tipo`    nas  classes    `ContaCorrente` , ContaPoupanca    e `ContaInvestimento`.  Faça  com  que  o   tipo    também  seja  impresso  quando  usamos  a  função
 `print()`.

# CAPÍTULO 10
## HERANÇA MÚLTIPLA E INTERFACES
Imagine que um Sistema de Controle do Banco pode ser acessado, além dos Gerentes, pelos Diretores do
Banco. Teríamos uma classe `Diretor` .

In [0]:
class Diretor(Funcionario):
    def autentica(self, senha):
        # verifica se a senha confere

E a classe  `Gerente`:

In [0]:
class Gerente(Funcionario):
    def autentica(self, senha):
        # verifica se a senha confere e também se o seu departamento tem acesso

Repare que o método de autenticação de cada tipo de   `Funcionario`  pode variar muito. Mas vamos aos  problemas.  Considere  o   `SistemaInterno`   e  seu  controle:  precisamos  receber  um   `Diretor`   ou
 `Gerente`  como argumento, verificar se ele se autentica e colocá-lo dentro do sistema.

Vimos  que  podemos  utilizar  a  função   `hasattr()`   para  verificar  se  um  objeto  possui  o  método
 `autentica()`:

In [0]:
class SistemaInterno:
    def login(self, funcionario):
        if(hasattr(obj, 'autentica')):
            # chama método autentica
        else:
            # imprime mensagem de ação inválida

Mas  podemos  esquecer,  no  futuro,  quando  modelar  a  classe   `Presidente` (que  também  é  um
funcionário  e  autenticável),  de  implementar  o  método   `autentica()`.  Não  faz  sentido  colocarmos  o
método  `autentica()`  na classe  `Funcionario`  já que nem todo funcionário é autenticável.

Uma  solução  mais  interessante  seria  criar  uma  classe  no  meio  da  árvore  de  herança,  a
 `FuncionarioAutenticavel`:

In [0]:
class FuncionarioAutenticavel(Funcionario):
    def autentica(self, senha):
        # verifica se a senha confere

E as classes   `Diretor`,   `Gerente`  e qualquer outro tipo de   `FuncionarioAutenticavel`  que vier a existir  em  nosso  sistema  bancário  passaria  a  estender  de   `FuncionarioAutenticavel`.  Repare  que `FuncionarioAutenticavel`  é forte candidata a classe abstrata. Mais ainda, o método   `autentica()` poderia ser um método abstrato.

O uso de herança simples resolve o caso, mas vamos a uma outra situação um pouco mais complexa:
todos os clientes também devem possuir acesso ao  `SistemaInterno` . O que fazer?
Uma opção é fazer uma herança sem sentido para resolver o problema, por exemplo, fazer  `Cliente` 
estender  de   `FuncionarioAutenticavel`.  Realmente  resolve  o  problema,  mas  trará  diversos  outros.
 `Cliente`  definitivamente não é um   `FuncionarioAutenticavel`. Se você fizer isso, o   `Cliente`  terá, por  exemplo,  um  método   `get_bonificacao()`,  um  atributo   `salario`   e  outros  membros  que  não fazem o menor sentido para esta classe.

Precisamos, para resolver este problema, arranjar uma forma de referenciar   `Diretor`,   `Gerente`  e
 `Cliente`  de uma mesma maneira, isto é, achar um fator comum.

Se  existisse  uma  forma  na  qual  essas  classes  garantissem  a  existência  de  um  determinado  método,
através de um contrato, resolveríamos o problema. Podemos criar um "contrato" que define tudo o que uma classe deve fazer se quiser ter um determinado status. Imagine:

    contrato Autenticavel
        * quem quiser ser Autenticavel precisa saber fazer:
            - autenticar dada uma senha, devolvendo um booleano

Quem  quiser  pode  assinar  este  contrato,  sendo  assim  obrigado  a  explicar  como  será  feita  essa
autenticação. A vantagem é que, se um   `Gerente`  assinar esse contrato, podemos nos referenciar a um
 `Gerente`  como um  `Autenticavel`.

Como Python admite herança múltipla podemos criar a classe  `Autenticavel`:

In [0]:
class Autenticavel:
    def autentica(self, senha):
        # verifica se a senha confere

E fazer  `Gerente`,  `Diretor`  e  `Cliente`  herdarem essa classe:

In [0]:
class Gerente(Funcionario, Autenticavel):
    # código omitido


In [0]:
class Diretor(Funcionario, Autenticavel):
    # código omitido


In [0]:
class Cliente(Autenticavel):
    # código omitido

Ou seja,  `Gerente`  e   `Diretor`  além de funcionários são autenticáveis! Assim, podemos utilizar o `SistemaInterno`  para funcionários autenticáveis e clientes:

In [0]:
class SistemaInterno:
    def login(self, obj):
        if(hasattr(obj, 'autentica')):
            obj.autentica()
            return True
        else:
            print('{} não é autenticável'.format(self.__class__.__name__))
            return False

In [0]:
diretor = Diretor('João', '111111111-11', 3000.0, '1234')
gerente = Gerente('José', '222222222-22', 5000.0, '1235')
cliente = Cliente('Maria', '333333333-33', '1236')
sistema = SistemaInterno()
sistema.login(diretor)
sistema.login(gerente)
sistema.login(cliente)

Note que uma classe pode herdar de muitas outras classes. Mas vamos aos problemas que isso pode gerar. Por exemplo, várias classes podem possuir o mesmo método.

### 10.1 PROBLEMA DO DIAMANTE
O  exemplo  anterior  pode  parecer  uma  boa  maneira  de  representar  classes  autenticáveis,  mas  se
começássemos  a  estender  esse  sistema,  logo  encontraríamos  algumas  complicações.  Em  um  banco  de
verdade,  as  divisões  entre  gerentes,  diretores  e  clientes  nem  sempre  são  claras.  Um   `Cliente`,  por
exemplo, pode ser um `Funcionario`, um   `Funcionario`   pode  ter  outras  subcategorias  como  fixos  e
temporários.

No Python, é possível que uma classe herde de várias outras classes. Poderíamos, por exemplo, criar
uma  classe   `A` ,  que  será  superclasse  das  classes   `B`   e   `C`.  A  herança  múltipla  não  é  muito  difícil  de entender se uma classe herda de várias classes que possuem propriedades completamente diferentes, mas as coisas ficam complicadas se duas superclasses implementam o mesmo método ou atributo.
Se as classes  `B`  e  `C`  herdarem a classe  `A`  e classe  `D`  herdar as classes   B  e   C , e as classes   `B`  e   `C`  têm um método  `m2()`, qual método a classe  `D`  herda?

In [0]:
class A:
    def m1(self):
        print('método de A')
class B(A):
    def m2(self):
        print('método de B')
class C(A):
    def m2(self):
        print('método de C')                
class D(B, C):
    pass

Essa ambiguidade é conhecida como o problema do diamante, ou problema do losango, e diferentes linguagens resolvem esse problema de maneiras diferentes. O Python segue uma ordem específica para percorrer a hierarquia de classes e essa ordem é chamada de MRO: Method Resolution Order (Ordem de Resolução de Métodos).

Toda  classe  tem  um  atributo   `__mro__`   que  retorna  uma  tupla  de  referências  das  superclasses  na ordem MRO - da classe atual até a classe  `object`. 

Vejamos o MRO da classe  `D`:

In [0]:
print(D.mro())

[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]


A ordem é sempre da esquerda para direita. Repare que o Python vai procurar a chamada do método `m2()`  primeiro na classe   `D`, não encontrando vai procurar em   `B`  (a  primeira classe  herdada). Caso não encontre em  `B`, vai procurar em  `C`  e só então procurar em  `A`  - e por último na classe  `object`.

Também podemos acessar o atributo  `__mro__`  através do método   `mro()`  chamado pela classe que retorna uma lista ao invés de uma tupla:

In [0]:
print(D.mro())

Portanto, seguindo o MRO, a classe  `D`  chama o método  `m2()`  da classe  `B`:

In [0]:
d = D()

In [0]:
d.m1()

In [0]:
d.m2()

Felizmente, a função  `super()`  sabe como lidar de forma inteligente com herança múltipla. Se usá-la dentro do método todos os métodos das superclasses devem ser chamados seguindo o MRO.

In [0]:
class A:
    def m1(self):
        print('método de A')
class B(A):
    def m1(self):
        super().m1()
    def m2(self):
        print('método de B')
class C(A):
    def m1(self):
        super().m1()
    def m2(self):
        print('método de C')                
class D(B, C):
    def m1(self):
        super().m1()
    def m2(self):
        super().m2()

In [0]:
d = D()
d.m1()
d.m2()

### 10.2 MIX-INS
Se  usarmos  herança  múltipla,  geralmente  é  uma  boa  idéia  projetarmos  nossas  classes  de  uma maneira que evite o tipo de ambiguidade descrita acima - apesar do Python possuir o MRO, em sistemas grandes a herança múltipla ainda pode causar muitos problemas.

Uma maneira de fazer isso é dividir a funcionalidade opcional em mix-ins. Um mix-in é uma classe que não se destina a ser independente - existe para adicionar funcionalidade extra a outra classe através de herança múltipla. A ideia é que classes herdem estes mix-ins, essas "misturas de funcionalidades".

Por  exemplo,  nossa  classe    `Autenticavel`    pode  ser  um  mix-in  já  que  ela  existe  apenas  para
acrescentar a funcionalidade de ser autenticável, ou seja, para herdar seu método  autentica .

Nossa classe  `Autenticavel`  já se comporta como um  Mix-In . No Python não existe uma maneira específica de criar mix-ins. Os programadores, por convenção e para deixar explícito a classe como um
mix-in, colocam o termo 'MixIn' no nome da classe e utilizam através de herança múltipla:

In [0]:
class AutenticavelMixIn:
    def autentica(self, senha):
        # verifica senha

Cada mix-in  é  responsável  por  fornecer  uma  peça  específica  de  funcionalidade  opcional.  Podemos
ter outros mix-ins no nosso sistema:

In [0]:
class AtendimentoMixIn:
    def cadastra_atendimento(self):
        # faz cadastro atendimento
    def atende_cliente(self):
        # faz atendimento
class HoraExtraMixIn:
    def calcula_hora_extra(self, horas):
        # calcula horas extras

E podemos misturá-los nas classes de nosso sistema:

In [0]:
class Gerente(Funcionario, AutenticavelMixIn, HoraExtraMixIn):
    pass

class Diretor(Funcionario, AutenticavelMixIn):
    pass

class Cliente(AutentivavelMixIn):
    pass    

class Escriturario(Funcionario, AtentimentoMixIn):
    pass

Repare  que  nossos  mix-ins  não  tem  um  método   `__init__()`.  Muitos  mix-ins  apenas  fornecem
métodos  adicionais  mas  não  inicializam  nada.  Isso  às  vezes  significa  que  eles  dependem  de  outras
propriedades que já existem em suas filhas. Cada mix-in é responsável por fornecer uma peça específica
de funcionalidade opcional - é um jeito de compor classes.

Poderíamos estender este exemplo com mais misturas que representam a capacidade de pagar taxas, a capacidade de ser pago por serviços, e assim por diante - poderíamos então criar uma hierarquia de classes relativamente plana para diferentes tipos de classes de funcionário que herdam   Funcionario  e alguns mix-ins .

Essa é uma das abordagens de se usar herança múltipla mas ela é bastante desencorajada. Caso você utilize,  opte  por  Mix  Ins  sabendo  de  suas  desvantagens.  Usado  em  sistemas  grandes  podem  ocorrer colisões com nomes de métodos, métodos substituídos acidentalmente, hierarquia de classe pouco clara e  dificuldade  de  ler  e  entender  classes  compostas  por  muitos  mix-ins,  dentre  outras  desvantagens.  O problema da herança múltipla permanece.

Outra  abordagem  possível  é  definir  funções  fora  de  classes,  digamos  em  um  módulo  e  fazer
chamadas  dessas  funções  passando  nossos  objetos.

Mas  isso  é  um  afastamento  radical  do  paradigma
orientado a objetos que é baseada em métodos definidos dentro das classes.

### 10.3 PARA SABE MAIS - TKINTER
Tkinter é um framework que faz parte da biblioteca padrão do Python utilizado para criar interface gráfica. É um caso onde mix-ins trabalham bem já que se trata de um pequeno framework, mas também é  suficientemente  grande  para  que  seja  possível  ver  o  problema.  Veja  um exemplo  de  parte  de  sua hierarquia de classe:

** imagem **

Essa  figura  mostra  parte  do  complicado  modelo  de  classes  utilizando  herança  múltipla  do  pacote
 Tkinter. A setas representam o MRO que deve iniciar na classe   `Text`. A classe   `Text`   implementa um campo de texto editável e tem muitas funcionalidades próprias, além de herdar muitos métodos de outras classes.

Uma outra classe do pacote que não aparece neste diagrama é a   `Label` , utilizada para mostrar um texto  ou  bitmap  na  tela.  Você  pode  testar  no  Pycharm,  aproveitando  a  ferramenta  de  autocomplete, chamando  `Tkinter.Label.`  e a IDE vai te mostrar 181 sugestões de atributos em uma única classe! Ou você pode utilizar a função  `help()`  para checar a origem de cada um deles.

In [0]:
from tkinter import *
help(Label)


Help on class Label in module tkinter:

class Label(Widget)
 |  Label widget which can display text and bitmaps.
 |  
 |  Method resolution order:
 |      Label
 |      Widget
 |      BaseWidget
 |      Misc
 |      Pack
 |      Place
 |      Grid
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, master=None, cnf={}, **kw)
 |      Construct a label widget with the parent MASTER.
 |      
 |      STANDARD OPTIONS
 |      
 |          activebackground, activeforeground, anchor,
 |          background, bitmap, borderwidth, cursor,
 |          disabledforeground, font, foreground,
 |          highlightbackground, highlightcolor,
 |          highlightthickness, image, justify,
 |          padx, pady, relief, takefocus, text,
 |          textvariable, underline, wraplength
 |      
 |      WIDGET-SPECIFIC OPTIONS
 |      
 |          height, state, width
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Bas

Esse pacote tem mais de 20 anos e é um exemplo de como a herança múltipla era utilizada quando os programadores  não  consideravam  suas  desvantagens.  Apesar  da  maioria  das  classes  se  comportarem como mix-ins, o padrão  de nomenclatura  não era utilizado.  Felizmente, o   `Tkinter` é  um  framework estável.

### 10.4 (OPCIONAL) EXERCÍCIOS - MIX-INS
1.  Nosso banco precisa tributar dinheiro de alguns bens que nossos clientes possuem. Para isso vamos criar uma classe  `Tributavel`:
        class Tributavel:
            def get_valor_imposto(self):
            pass
Lemos essa classe da seguinte maneira: "Todos que quiserem ser tributável precisam saber retornar o
valor do imposto".  Alguns bens  são tributáveis e  outros não,   `ContaPoupanca`   não  é  tributável,  já
para  `ContaCorrente`  você precisa pagar `1%` da conta e o   `SeguroDeVida`  tem uma faixa fixa de `50`
reais mais `5%` do valor do seguro.

2.  Torne a classe  `Tributavel` um mix-in:
        class TributavelMixIn:
            def get_valor_imposto(self)
            pass

3.  Faça a classe  `ContaCorrente`  herdar da classe  `TributavelMixIn`. Crie a classe  `SeguroDeVida`:
        class ContaCorrente(Conta, TributavelMixIn):
            # código omitido
            def get_valor_imposto(self):
                return self._saldo * 0.01
        class SeguroDeVida(TributavelMixIn):
            def __init__(self, valor, titular, numero_apolice):
                self._valor = valor
                self._titular = titular
                self._numero_apolice = numero_apolice
                def get_valor_imposto(self):
                return 42 + self._valor * 0.05
4.  Vamos criar a classe   `ManipuladorDeTributaveis`  em um arquivo chamado `manipulador.py`. Essa
classe  deve  ter  um  método  chamado   `calcula_imposto()`   que  recebe  umas  lista  de  tributáveis  e
retorna o total de impostos cobrados:
        class ManipuladorDeTributaveis:
            def calcula_impostos(self, lista_tributaveis):
            total = 0
            for t in lista_tributaveis:
                total += t.get_valor_imposto()
            return total

5.  Ainda no arquivo  `manipulador.py` , vamos testar o código. Crie alguns objetos de  `ContaCorrente` 
e de   `SeguroDeVida`. Em seguida, crie uma lista de tributáveis e insira seus objetos nela. Instancie um   `ManipuladorDeTributaveis`   e  chame  o  método   `calcula_impostos()`   passando  a  lista  de
tributáveis criada e imprima o valor total dos impostos:

        from conta import ContaCorrente, SeguroDeVida, TributavelMixIn
        cc1 = ContaCorrente('123-4', 'João', 1000.0)
        cc2 = ContaCorrente('123-4', 'José', 1000.0)
        seguro1 = SeguroDeVida(100.0, 'José', '345-77')
        seguro2 = SeguroDeVida(200.0, 'Maria', '237-98')
        lista_tributaveis = []
        lista_tributaveis.append(cc1)
        lista_tributaveis.append(cc2)
        lista_tributaveis.append(seguro1)
        lista_tributaveis.append(seguro2)
        manipulador = ManipuladorDeTributaveis()
        total = manipulador.calcula_impostos(lista_tributaveis)
        print(total)
Vimos que herança múltipla pode ser perigosa e se nosso sistema crescer pode gerar muita confusão e  conflito  de  nomes  de  métodos.  Uma  maneira  mais  eficaz  nestes  casos  é  usar  classes  abstratas  como
interfaces que veremos a seguir.

### 10.5 INTEFACES
O  Python  não  possui  uma  palavra  reservada  interface.  Mesmo  sem  uma  palavra  reservada  para
interface toda classe  tem uma  interface. São os  atributos públicos  definidos (que em  Python são tanto
atributos  quanto  métodos)  em  uma  classe  -  isso  inclui  os  métodos  especiais  como   `__str__()` e
 `__add__()`.

Uma  interface  vista  como  um  conjunto  de  métodos  para  desempenhar  um  papel  é  o  que  os programadores da SmallTalk chamavam de protocolo e este termo foi disseminado em comunidades de programadores de linguagens dinâmicas. Esse protocolo funciona como um contrato.

Os protocolos são independentes de herança. Uma classe pode implementar vários protocolos, como os  mix-ins.  Protocolos  são  interfaces  e  são  definidos  apenas  por  documentação  e  convenções  em linguagens  dinâmicas,  por  isso  são  considerados  informais.  Os  protocolos  não  podem  ser  verificados estaticamente pelo interpretador.

O método  `__str__()`, por exemplo, é esperado que retorne uma representação do objeto em forma de `string` . Nada impede de fazermos outras coisas dentro do método como deletar algum conteúdo, fazer algum cálculo, etc... ao invés de retornarmos apenas a `string`. Mas há um entendimento prévio comum do que este método deve fazer e está presente na documentação do Python. Este é um exemplo
onde  o  contrato  semântico  é  descrito  em  um  manual.  Algumas  linguagens  de  tipagem  estática,  como
Java,  possuem  interfaces  em  sua  biblioteca  padrão  e  podem  garantir  este  contrato  em  tempo  de compilação.

A  partir  do  Python  2.6  a  definição  de  interfaces  utilizando  o  módulo  `ABC`  é  uma  solução  mais
elegante do que os mix-ins. Nossa classe   Autenticavel   pode  ser  uma  classe  abstrata  com  o  método
abstrato  `autentica()`:

In [0]:
import abc
class Autenticavel(abc.ABC):
    @abc.abstractmethod
    def autentica(self, senha):
        pass

Como  se  trata  de  uma  interface  em  uma  linguagem  de  tipagem  dinâmica  como  o  Python,  a  boa
prática é documentar esta classe garantindo o contrato semântico:

In [0]:
import abc
class Autenticavel(abc.ABC):
    """Classe abstrata que contém operações de um objeto autenticável.
    As subclasses concretas devem sobrescrever o método autentica
    """
    @abc.abstractmethod
    def autentica(self, senha):
        """ Método abstrato que faz verificação da senha
        return True se a senha confere, e False caso contrário.
        """

E nossas classes   `Gerente`,   `Diretor` e `Cliente` herdariam a classe   `Autenticavel`. Mas qual a
diferença de herdar muitos mix-ins e muitas `ABC`s? Realmente, aqui não há grande diferença e voltamos
ao problema anterior dos mix-ins - muito acoplamento entre classes que gera a herança múltipla.

Mas a novidade das ABCs é seu método   `register()`. As ABCs introduzem uma subclasse virtual, que são classes que não herdam de uma classe mas são reconhecidas pelos métodos   `isinstance()`   e `issubclass()`.  Ou  seja,  nosso    `Gerente`    não  precisa  herdar  a  classe    `Autenticavel`,  basta registrarmos ele como uma implementação da classe  `Autenticavel`.

In [0]:
Autenticavel.register(Gerente)

E testamos os métodos  `isinstance()`  e  `issubclass()`  com uma instância de  `Gerente`:

In [0]:
gerente = Gerente('João', '111111111-11', 3000.0)

In [0]:
print(isinstance(Autenticavel))

In [0]:
print(issubclass(Autenticavel))

O  Python  não  vai  verificar  se  existe  uma  implementação  do  método   `autentica()`   em   `Gerente` quando  registrarmos  a  classe.  Ao  registrarmos  a  classe    `Gerente`    como  uma    `Autenticavel`, prometemos ao Python que a classe implementa fielmente a nossa interface  `Autenticavel`  definida. O Python vai acreditar nisso e retornar `True` quando os métodos  `isinstance()`  e  `issubclass()`  forem chamados. Se mentirmos, ou seja, não implementarmos o método   `autentica()`  em   `Gerente`,  uma exceção será lançada quando tentarmos chamar este método.

Vejamos um exemplo com a classe   `Diretor`. Não vamos implementar o método   `autentica()`  e registrar uma instância de  `Diretor`  como um  `Autenticavel`:

In [0]:
class Diretor(Funcionario):
    # código omitido

In [0]:
Autenticavel.register(Diretor)

In [0]:
d = Diretor('José', '22222222-22', 3000.0)

In [0]:
d.autentica('?')

Novamente, podemos tratar a exceção ou utilizar os métodos   `isinstance()`   ou   `issubclass()` para  verificação.  Apesar  de  considerada  má  práticas  por  muitos  pythonistas,  o  módulo  de  classes abstratas justifica a utilização deste tipo de verificação. A verificação não é de tipagem, mas se um objeto está de acordo com a interface:

In [0]:
Autenticavel.register(Diretor)
d = Diretor('José', '22222222-22', 3000.0)
if(isinstance(d, Autenticavel)):
    d.autentica('?')
else:
    print("Diretor não implementa a interface Autenticavel")

e portanto, nossa classe  `SistemaInterno` ficaria assim:

In [0]:
from autenticavel import Autenticavel
class SistemaInterno:
    def login(self, obj):
        if(isinstance(obj, Autenticavel)):
            obj.autentica(obj.senha)
            return True
        else:
            print("{} não é autenticável".format(self.__class__.__name__))
            return False

Dessa  maneira  fugimos  da  herança  múltipla  e  garantimos  um  contrato,  um  protocolo.  Classes abstratas  complementam  o  duck  typing  provendo  uma  maneira  de  definir  interfaces  quando  técnicas como usar  `hasattr()`  são ruins ou sutilmente erradas. Você pode ler mais a respeito no documento da
PEP  que  introduz  classes  abstratas  -  é  a  PEP  3119  e  você  pode  acessar  seu  conteúdo  neste  link:
https://www.python.org/dev/peps/pep-3119/

Algumas  `ABC`s  também  podem  prover  métodos  concretos,  ou  seja,  não  abstratos.  Por  exemplo,  a
classe   `Interator`   do  módulo   collections   da  biblioteca  padrão  do  Python  possui  um  método
 `__iter__()`  retornando ele mesmo. Esta ABC pode ser considerada uma classe mix-in.

O Python já vem com algumas estruturas abstratas (ver módulos `collections`, `numbers` e `io`).

### 10.6 EXERCÍCIOS - INTERFACES E CLASSES ABSTRATAS
1.  Nosso banco precisa tributar dinheiro de alguns bens que nossos clientes possuem. Para isso vamos criar uma classe  `Tributavel`  no módulo `tributavel.py`:
        class Tributavel:
            def get_valor_imposto(self):
            pass
Lemos essa classe da seguinte maneira: "Todos que quiserem ser tributável precisam saber retornar o
valor do imposto".  Alguns bens  são tributáveis e  outros não,   `ContaPoupanca`   não  é  tributável,  já
para  `ContaCorrente`  você precisa pagar `1%` da conta e o   `SeguroDeVida`  tem uma faixa fixa de `50`
reais mais `5%` do valor do seguro.

2.  Torne a classe  `Tributavel`  uma classe abstrata:
        import abc
        class Tributavel(abc.ABC):
            def get_valor_imposto(self)
                pass

3.  O método  `get_valor_imposto()`  também deve ser abstrato:
import abc
        class Tributavel(abc.ABC):
            @abc.abstractmethod
            def get_valor_imposto(self, valor):
                pass

4.  Nada impede que os usuários de nossa classe tributavel implemente o método  `get_valor_imposto` 
de maneira não esperada por nós. Então vamos acrescentar a documentação utilizando docstring que aprendemos no capítulo de módulos:

        import abc
        class Tributavel(abc.ABC):
            """ Classe que contém operações de um objeto autenticável
            As subclasses concretas devem sobrescrever o método get_valor_imposto.
            """
           @abc.abstractmethod
            def get_valor_imposto(self):
                """ aplica taxa de imposto sobre um determinado valor do objeto """
                pass

5.  Utiliza a função  `help()`  passando a classe  `Tributavel`  para acessar a documentação.

6.  Faça a classe  `ContaCorrente`  herdar da classe   `Tributavel`. Crie a classe   `SeguroDeVida`  com os
atributos  `valor`,  `titular` e  `numero_apolice`  que também deve ser um tributável. Implemente o método  `get_valor_imposto()`  de acordo com a regra de negócio definida pelo exercício:
        class ContaCorrente(Conta, Tributavel):
            # código omitido
            def get_valor_imposto(self):
                return self._saldo * 0.01
        class SeguroDeVida(Tributavel):
            def __init__(self, valor, titular, numero_apolice):
                self._valor = valor
                self._titular = titular
                self._numero_apolice = numero_apolice
            def get_valor_imposto(self):
                return 50 + self._valor * 0.05

7.  Vamos  criar  a  classe `ManipuladorDetributaveis`   em  um  arquivo  chamado `manipulador_tributaveis.py`.  Essa  classe  deve  ter  um  método  chamado   `calcula_imposto()` que recebe umas lista de tributáveis e retorna o total de impostos cobrados:
        class ManipuladorDeTributaveis:
            def calcula_impostos(self, lista_tributaveis):
                total = 0
                for t in lista_tributaveis:
                    total += t.get_valor_imposto()
                return total

8.  Nossas  classes    `ContaCorrente`    e    `SeguraDeVida`    já  implementam  o  método
 `get_valor_imposto()`. Vamos instanciar cada umas delas e testar a chamada do método:
        if __name__ == '__main__':
            cc = ContaCorrente('123-4', 'João', 1000.0)
            seguro = SeguroDeVida(100.0, 'José', '345-77')
            print(cc.get_valor_imposto())
            print(seguro.get_valor_imposto())

9.  Crie  uma  lista  com  os  objetos  criados  no  exercício  anterior,  instancie  um  objeto  do  tipo   `list`   e passe a lista chamando o método  `calcula_impostos()`.
        if __name__ == '__main__':
            # código omitido
            lista_tributaveis = []
            lista_tributaveis.append(cc)
            lista_tributaveis.append(seguro)
            mt = ManipuladorDeTributaveis()
            total = mt.calcula_impostos(lista_tributaveis)
            print(total)

10.  Nosso  código  funciona,  mas  ainda  estamos  utilizando  herança  múltipla!  Vamos  melhorar  nosso
código. Faça com que  `ContaCorrente`  e  `SeguroDeVida`  não mais herdem da classe   `Tributavel`.
Vamos  registrar  nossas  classes   `ContaCorrente`   e   `SeguroDeVida`   como  subclasses  virtuais  de
 `Tributavel`, de modo que funcione como uma interface.
            class ContaCorrente(Conta):
            # código omitido
            class SeguroDeVida:
            # código omitido
            if __name__ == '__main__':
            from tributavel import Tributavel
            cc = ContaCorrente('João', '123-4')
            cc.deposita(1000.0)
            seguro = SeguroDeVida(100.0, 'José', '345-77')
            Tributavel.register(ContaCorrente)
            Tributavel.register(SeguroDeVida)
            lista_tributaveis = []
            lista_tributaveis.append(cc)
            lista_tributaveis.append(seguro)
            mt = ManipuladorDeTributaveis()
            total = mt.calcula_impostos(lista_tributaveis)
            print(total)

11.  Modifique o método  `calcula_impostos()`  da classe  `ManipuladorDeTributaveis`  para checar se
os  elementos  da  listas  são  tributáveis  através  do  método   `isinstance()`.  Caso  um  objeto  da  lista
não  seja  um  tributável,  vamos  imprimir  uma  mensagem  de  erro  e  apenas  os  tributáveis  serão
somados ao total:

        class ManipuladorDeTributaveis:
            def calcula_impostos(self, lista_tributaveis):
                total = 0
                for t in lista_tributaveis:
                    if(isinstance(t, Tributavel)):
                        total += t.get_valor_imposto()
                    else:
                        print(t.__repr__(), "não é um tributável")    
               return total
Teste novamente com a lista de tributáveis que fizemos no exercício anterior e veja se tudo continua funcionando.

12.   `ContaPoupanca`   não  é  um  tributável.  Experimente  instanciar  uma   `ContaPoupanca`,  adicionar  a lista de tributáveis e calcular o total de impostos através do  `ManipuladorDetributaveis`:
        if __name__ == '__main__':
            #código omitido do exercício anterior omitido
            cp = ContaPoupanca('123-6', 'Maria')
            lista_tributaveis.append(cp)
            total = mt.calcula_impostos(lista_tributaveis)
            print(total)
O que acontece?

13.  (Opcional) Agora além de  `ContaCorrente`  e  `SeguroDeVida`  nossa   `ContaInvestimento`  também deve  ser  um  tributável,  cobrando  `3%`  do  saldo.  Instancie  uma   `ContaInvestimento`   e  registre  a
classe   `ContaInvestimento`   como  tributável.  Adicione  a   `ContaInvestimento`   criada  na  lista  de
tributáveis do exercício anterior e calcule o total de impostos através do `ManipuladorDeTributaveis`.

Neste capítulo aprendemos sobre herança múltipla e suas desvantagens mesmo utilizando mix-ins.

Aprendemos  utilizar  classes  abstratas  como  interfaces  registrando  as  classes  e  evitando  os  problemas com  a  herança  múltipla.  Agora  nossa  classe  abstrata   `Tributavel`   funciona  como  um  protocolo.  No capítulo sobre o módulo `collections` veremos na prática alguns conceitos vistos nestes capítulo.

# CAPÍTULO 11
## EXCEÇÕES E ERROS
Voltando as contas que criamos no capítulo 6, o que aconteceria ao tentar chamar o método   saca() 
com um valor fora do limite? O sistema mostraria uma mensagem de erro, mas quem chamou o método
 saca()  não saberá que isso aconteceu.
Como avisar aquele que chamou o método de que ele não conseguiu fazer aquilo que deveria?
Os métodos dizem qual o contrato que eles devem seguir. Se, ao tentar   sacar() , ele não consegue
fazer o que deveria, ele precisa, ao menos, avisar ao usuário que o saque não foi feito.
Veja no exemplo abaixo: estamos forçando uma  Conta  a ter um valor negativo, isto é, estar em um
estado inconsistente de acordo com a nossa modelagem.

    conta = Conta('123-4', 'João')
    conta.deposita(100.0)
    conta.saca(3000.0)
    #o método saca funcionou?

Em  sistemas  de  verdade,  é  muito  comum  que  quem  saiba  tratar  o  erro  é  aquele  que  chamou  o
método e não a própria classe! Portanto, nada mais natural sinalizar que um erro ocorreu.
A solução mais simples utilizada antigamente é a de marcar o retorno de um método como boolean e
retornar  True , se tudo ocorreu da maneira planejada, ou  False , caso contrário:

    if (valor > self.saldo + self.limite):
        print("nao posso sacar fora do limite")
        return False
    else:
        self.saldo -= valor
        return True

Um novo exemplo de chamada do método acima:

    conta = Conta('123-4', 'João')
    conta.deposita(100.0)
    conta.limite = 100.0
    if(not conta.saca(3000.0)):
        print("nao saquei")

Repare que tivemos de lembrar de testar o retorno do método, mas não somos obrigados a fazer isso.
Esquecer de testar o retorno desse método teria consequências drásticas: a máquina de autoatendimento
poderia vir a liberar a quantia desejada de dinheiro, mesmo se o sistema não tivesse conseguido efetuar o
método  saca()  com sucesso, como no exemplo a seguir:

    conta = Conta("123-4", "João")
    conta.deposita(100.0)
    # ...
    valor = 5000.0
    conta.saca(valor) # vai retornar False, mas ninguém verifica
    caixa_eletronico.emite(valor)

Mesmo  invocando  o  método  e  tratando  o  retorno  de  maneira  correta,  o  que  faríamos  se  fosse
necessário sinalizar quando o usuário passou um valor negativo como valor. Uma solução seria alterar o
retorno de   boolean  para   int  e retornar o código do erro que ocorreu. Isso é considerado uma má
prática (conhecida também como uso de "magic numbers").
Além de você perder o retorno do método, o valor devolvido é "mágico" e só legível perante extensa
documentação, além de não obrigar o programador a tratar esse retorno e, no caso de esquecer isso, seu
programa continuará rodando já num estado inconsistente.
Por  esses  e  outro  motivos,  utilizamos  um  código  diferente  para  tratar  aquilo  que  chamamos  de
exceções: os casos onde acontece algo que, normalmente, não iria acontecer. O exemplo do argumento
do saque inválido ou do id inválido de um cliente é uma exceção à regra.
Uma exceção representa uma situação que normalmente não ocorre e representa algo de estranho ou
inesperado no sistema.
Antes  de  resolvermos  o  nosso  problema,  vamos  ver  como  o  interpretador  age  ao  se  deparar  com
situações inesperadas, como divisão por zero ou acesso a um índice de uma lista que não existe.
Para  aprendermos  os  conceitos  básicos  das  exceptions  do  Python,  crie  um  arquivo  teste_erro.py  e
teste o seguinte código você mesmo:

In [0]:
def metodo1():
    print('início do metodo1')
    metodo2()
    print('fim do metodo1')
def metodo2():
    print('início do metodo2')
    cc = ContaCorrente('José', '123')
    for i in range(1,15):
        cc.deposita(i + 1000)
        print(cc.saldo)
        if(i == 5):
            cc = None
    print('fim do metodo2')
if __name__ == '__main__':
    print('início do main')
    metodo1()
    print('fim do main')

Repare que durante a execução do programa chamamos o   metodo1()  e esse, por sua vez, chama o
 metodo2() . Cada um desses métodos pode ter suas próprias variáveis locais, isto é: o   metodo1()  não
enxerga as variáveis declaradas dentro do executável e por aí em diante.
Como o Python (e muitas outras linguagens) faz isso? Toda invocação de método é empilhado em
uma estrutura de dados que isola a área e memória de cada um. Quando um método termina (retorna),
ele  volta  para  o  método  que  o  invocou.  Ele  descobre  isso  através  da  pilha  de  execução  (stack):  basta
remover o marcador que está no topo da pilha:
Porém,  o  nosso   metodo2()   propositalmente  possui  um  enorme  problema:  está  acessando  uma
referência para  None  quando o índice for igual a 6!
Rode o código. Qual a saída? O que isso representa? O que ela indica?
Essa saída é o rastro de pilha, o Traceback. É uma saída importantíssima para o programador - tanto
que, em qualquer fórum ou lista de discussão, é comum os programadores enviarem, juntamente com a
descrição do problema, essa Traceback. Mas por que isso aconteceu?
O  sistema  de  exceções  do  Python  funciona  da  seguinte  maneira:  quando  uma  exceção  é  lançada
(raise), o interpretador entra em estado de alerta e vai ver se o método atual toma alguma precaução ao
tentar executar esse trecho de código. Como podemos ver, o   metodo2()  não toma nenhuma medida
diferente do que vimos até agora.
Como  o    metodo2()    não  está  tratando  esse  problema,  o  interpretador  para  a  execução  dele
anormalmente,  sem  esperar  ele  terminar,  e  volta  um  stackframe  para  baixo,  onde  será  feita  nova
verificação: "o   método1()  está se precavendo de um problema chamado   AttributeError ?  "Não..."
Volta para o executável, onde também não há proteção, então o interpretador morre.
Obviamente, aqui estamos forçando esse caso e não faria sentido tomarmos cuidado com ele. É fácil
arrumar  um  problema  desses:  basta  verificar  antes  de  chamar  os  métodos  se  a  variável  está  com
referência para  None .
Porém, apenas para entender o controle de fluxo de uma Exception, vamos colocar o código que vai
tentar  (try)  executar  um  bloco  perigoso  e,  caso  o  problema  seja  do  tipo   AttributeError ,  ele  será
excluído(except).  Repare  que  é  interessante  que  cada  exceção  no  Python  tenha  um  tipo...  ela  pode  ter
atributos e métodos.
Adicione um   try/except   em  volta  do   for ,  'pegando'  um   AttributeError .  O  que  o  código
imprime?

In [0]:
from conta import ContaCorrente
def metodo1():
    print('início do metodo1')
    metodo2()
    print('fim do metodo1')
def metodo2():
    print('início do metodo2')
    cc = ContaCorrente('José', '123')
    try:
        for i in range(1,15):
            cc.deposita(i + 1000)
            print(cc.saldo)
            if(i == 5):
            cc = None
    except:
        print('erro')
    print('fim do metodo2')
if __name__ == '__main__':
    print('início do main')
    metodo1()
    print('fim do main')

Em vez de fazer o  try  em torno do  for  inteiro, tente apenas com o bloco dentro do  for :

In [0]:
def metodo2():
    print('início do metodo2')
    cc = ContaCorrente('José', '123')
    for i in range(1,15):
        try:
            cc.deposita(i + 1000)
            print(cc.saldo)
            if(i == 5):
                cc = None
        except:
            print('erro')
    print('fim do metodo2')

Qual a diferença?
Retire o  try/except  e coloque ele em volta da chamada do  metodo2() :

In [0]:
def metodo1():
    print('início do metodo1')
    try:
        metodo2()
    except AttributeError:
        print('erro')   
    print('fim do metodo1')

Faça  o  mesmo,  retirando  o    try/except    novamente  e  colocando  em  volta  da  chamada  do
 metodo1() . Rode os códigos, o que acontece?

In [0]:
if __name__ == '__main__':
    print('início do main')
    try:
        metodo1()
    except AttributeError:
        print('erro')   
    print('fim do main')

Repare que, a partir do momento que uma exception foi catched (pega, tratada, handled), a exceção
volta ao normal a partir daquele ponto.

## 11.1 EXCEÇÕES E TIPOS DE ERROS

### Runtime
Este tipo de erro ocorre quando algo de errado acontece durante a execução do programa. A maior
parte das mensagens deste tipo de erro inclui informações do que o programa estava fazendo e o local
que o erro aconteceu.
O interpretador mostra a famosa Traceback - ele mostra a sequência de chamadas de função que fez
com  que  você  chegasse  onde  está,  incluindo  o  número  da  linha  de  seu  arquivo  onde  cada  chamada
ocorreu.
Os erros mais comuns de tempo de execução são:
#### NameError
Quando tentamos acessar uma variável que não existe.

    print(x)
    Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    NameError: name 'x' is not defined

No exemplo acima tentamos imprimir  x  sem defini-lo antes. Este erro também é muito comum de
ocorrer quando tentamos acessar um variável local em um contexto local.
#### TypeError
Quando  tentamos  usar  um  valor  de  forma  inadequada,  como  por  exemplo  tentar  indexar  um
sequência com algo diferente de um número inteiro ou de um fatiamento:

    lista = [1, 2, 3]
    print(lista['a'])
    Traceback (most recent call last):
    File "<stdin>", line 2, in <module>
    TypeError: list indices must be integers or slices, not str


#### KeyError
Quando tentamos acessar um elemento de um dicionário usando uma chave que não existe.

    dicionario = {'nome': 'João', 'idade': 25}
    print(dicionario['cidade'])
    Traceback (most recent call last):
    File "<stdin>", line 2, in <module>
    KeyError: 'cidade'

#### AttributeError
Quando tentamos acessar um atributo ou método que não existe em um objeto.

    lista = [1, 2, 3]
    print(lista.nome)
    Traceback (most recent call last):
    File "<stdin>", line 2, in <module>
    AttributeError: 'list' object has no attribute 'nome'

#### IndexError
Quando  tentamos  acessar  um  elemento  de  uma  sequência  com  um  índice  maior  que  seu
comprimento menos um.

    tupla = (1, 2, 3)
    print(tupla[3])
    Traceback (most recent call last):
    File "<stdin>", line 2, in <module>
    IndexError: tuple index out of range

## 11.2 TRATANDO EXCEÇÕES
Há  muitos  outros  erros  de  tempo  de  execução.  Que  tal  dividir  um  número  por  zero?  Será  que  o
interpretador consegue fazer aquilo que 
nós definimos que não existe?

    n = 2
    n = n / 0
    Traceback (most recent call last):
    File "<stdin>", line 2, in <module>
    ZeroDivisionError: division by zero

Repare que um  ZeroDivisionError  poderia ser facilmente evitado com um   
if  que checaria se o
denominador  é  diferente  de  zero  mas  a  forma  correta  de  se  tratar  um  erro  no  Python  é  através  do
comando try/except:

    try:
        n = n/0
    except ZeroDivisionError:
        print('divisão por zero')

Que gera a saída:

    divisão por zero

O conjunto de instruções dentro do bloco   try  é executado (o interpretador tentará executar), se
nenhuma  exceção  ocorrer,  o  comando   except   é  ignorado  e  a  execução  é  finalizada.  Mas  se  ocorrer
alguma  exceção  durante  a  execução  do  bloco   try ,  as  instruções  remanescentes  são  ignoradas  e  se  a
exceção lançada prever um  except , então as instruções dentro do bloco  except  são executadas.
O comando  try  pode ter mais de um comando  except  para especificar múltiplos tratadores para
diferentes exceções. No máximo um único tratador será ativado. Tratadores só são sensíveis às exceções
levantadas  no  interior  da  cláusula   try ,  e  não  as  que  tenham  ocorrido  no  interior  de  outro  tratador
numa  mesma  instrução    try  .  Um  tratador  pode  ser  sensível  a  múltiplas  exceções,  desde  que  as
especifique em uma tupla:

    except(RuntimeError, TypeError, NameError):
        pass

A última cláusula   except  pode omitir o nome da exceção, funcionando como um curinga. Não é
aconselhável abusar deste recurso já que isso pode esconder erros do programador e do usuário.
O bloco   try/except  possui um comando opcional   else  que, quando usado, deve ser colocado
depois de todos os comandos  except . É útil para código que precisa ser executado se nenhuma exceção
foi lançada, por exemplo:

    try:
        arquivo = open('palavras.txt', 'r')
    except IOError:
        print('não foi possível abrir o arquivo')
    else:
        print('o arquivo tem {} palavras'.format(len(arquivo.readlines())))
        arquivo.close()

## 11.3 LEVANTANDO EXCEÇÕES
O  comando   raise   nos  permite  forçar  a  ocorrência  de  um  determinado  tipo  de  exceção.  Por
exemplo:

    raise NameError('oi')
    Traceback (most recent call last):
    File "<stdin>", line 1, in ?
    NameError: oi

O argumento de   raise  indica a exceção a ser lançada. Esse argumento deve ser uma instância de
 Exception  ou uma classe de alguma exceção - uma classe que deriva de  Exception .
Caso você precise determinar se uma exceção foi lançada ou não, mas não quer manipular o erro,
uma forma é lançá-la novamente através da instrução  raise :

    try:
        raise NameError('oi')
    except NameError:
        print('lançou uma exceção')
        raise

Saída:

    lançou uma exceção        
    Traceback (most recent call last):
    File "<stdin>", line 1, in ?
    NameError: oi

## 11.4 DEFINIR UMA EXCEÇÃO
Programas podem definir novos tipos de exceções, através da criação de uma nova classe. Exceções
devem ser derivadas da classe  Exception , direta ou indiretamente. Por exemplo:

    class MeuErro(Exception):
        def __init__(self, valor):
            self.valor = valor
        def __str__(self):
            return repr(self.valor)
    if __name__ == '__main__':
        try:
            raise MeuErro(2*2)
        except MeuErro as e:
            print('Minha exceção ocorreu, valor: {}'.format(e.valor))
        raise MeuErro('oops!')

Que quando executado gera a saída:

    Minha exceção ocorreu, valor: 4
    Traceback (most recent call last):
    File "<stdin>", line 13, in <module>
        raise MeuErro('oops!')
        __main__.MeuErro: 'oops!'

Neste exemplo, o método   `__init__`  da classe   Exception  foi  reescrito. O  novo comportamento
simplesmente  cria  o  atributo  valor.  Classes  de  exceções  podem  ser  definidas  para  fazer  qualquer  coisa
que qualquer outra classe faz, mas em geral são bem simples, frequentemente oferecendo apenas alguns
atributos que fornecem informações sobre o erro que ocorreu.
Ao criar um módulo que pode gerar diversos erros, uma prática comum é criar uma classe base para
as  exceções  definidas  por  aquele  módulo,  e  as  classes  específicas  para  cada  condição  de  erro  como
subclasses dela:

    class MeuError(Exception):
        """Classe base para outras exceções"""
        pass
    class ValorMuitoPequenoError(Error):
        """É lançada quando o valor passado é muito pequeno"""
        pass
    class ValorMuitoGrandeError(Error):
        """É lançada quando o valor passado é muito grande"""
        pass

Essa é a maneira padrão de definir exceções no Python mas o programador não precisa ficar preso a
ela.  É  comum  que  novas  exceções  sejam  definidas  com  nomes  terminando  em  “Error”,  semelhante  a
muitas exceções embutidas.

## 11.5 PARA SABER MAIS: FINALLY
O  comando  try  pode  ter  outro  comando  opcional  chamado  finally.  Sua  finalidade  é  permitir  a
implementação  de  ações  de  limpeza,  que  sempre  devem  ser  executadas  independentemente  da
ocorrência de exceções. Como no exemplo:

    def divisao(x, y):
        try:
            resultado = x / y
        except ZeroDivisionError:
            print("Divisão por zero")
        else:
            print("o resultado é {}".format(resultado))
        finally:
            print("executando o finally")
    if __name__ == '__main__':
        divide(2, 1)
        divide(2, 0)
        divide('2', '1')

Executando:

    resultado é 2
    executando o finally
    divisão por zero
    executando o finally
    executando o finally
    Traceback (most recent call last):
    File "<stdin>", line 1, in ?
    File "<stdin>", line 3, in divide
    TypeError: unsupported operand type(s) for /: 'str' and 'str'

Repare  que  o  bloco  finally  é  executado  em  todos  os  casos.  A  exceção   TypeError   levantada  pela
divisão de duas strings e não é tratada no except e portanto é relançada depois que o finally é executado.
Em  aplicações  reais,  o  finally  é  útil  para  liberar  recursos  externos  (como  arquivos  ou  conexões  de
rede), independentemente do uso do recurso ter sido bem sucedido ou não.

## 11.6 ÁRVORE DE EXCEÇÕES
No  Python  todas  as  exceções  são  instâncias  de  uma  classe  derivada  de   BaseException .  Ela  não
serve  para  ser  diretamente  herdada  por  exceções  criadas  por  programadores,  para  isso  utilizamos
 Exception  que também é filha de  BaseException .
Abaixo está a hierarquia de classes de exceções do Python. Para mais informações sobre cada uma
delas consulte a documentação: https://docs.python.org/3/library/exceptions.html

    BaseException
    +-- SystemExit
    +-- KeyboardInterrupt
    +-- GeneratorExit
    +-- Exception
        +-- StopIteration
        +-- StopAsyncIteration
        +-- ArithmeticError
        |    +-- FloatingPointError
        |    +-- OverflowError
        |    +-- ZeroDivisionError
        +-- AssertionError
        +-- AttributeError
        +-- BufferError
        +-- EOFError
        +-- ImportError
        |    +-- ModuleNotFoundError
        +-- LookupError
        |    +-- IndexError
        |    +-- KeyError
        +-- MemoryError
        +-- NameError
        |    +-- UnboundLocalError
        +-- OSError
        |    +-- BlockingIOError
        |    +-- ChildProcessError
        |    +-- ConnectionError
        |    |    +-- BrokenPipeError
        |    |    +-- ConnectionAbortedError
        |    |    +-- ConnectionRefusedError
        |    |    +-- ConnectionResetError
        |    +-- FileExistsError
        |    +-- FileNotFoundError
        |    +-- InterruptedError
        |    +-- IsADirectoryError
        |    +-- NotADirectoryError
        |    +-- PermissionError
        |    +-- ProcessLookupError
        |    +-- TimeoutError
        +-- ReferenceError
        +-- RuntimeError
        |    +-- NotImplementedError
        |    +-- RecursionError
        +-- SyntaxError
        |    +-- IndentationError
    .
        |         +-- TabError
        +-- SystemError
        +-- TypeError
        +-- ValueError
        |    +-- UnicodeError
        |         +-- UnicodeDecodeError
        |         +-- UnicodeEncodeError
        |         +-- UnicodeTranslateError
        +-- Warning
            +-- DeprecationWarning
            +-- PendingDeprecationWarning
            +-- RuntimeWarning
            +-- SyntaxWarning
            +-- UserWarning
            +-- FutureWarning
            +-- ImportWarning
            +-- UnicodeWarning
            +-- BytesWarning
            +-- ResourceWarning

### 11.7 EXERCÍCIOS: EXCEÇÕES
1.  Na  classe   `Conta`,  modifique  o  método   `deposita()`.  Ele  deve  lançar  uma  exceção  chamada `ValueError`, que já  faz parte  da biblioteca padrão  do Python,  sempre que o  valor passado  como argumento for inválido (por exemplo, quando for negativo):
 
        def deposita(self, valor):
            if(valor < 0):
                raise ValueError
            else:
                self._saldo += valor
2.  Da  maneira  com  está,  apenas  saberemos  que  ocorreu  um   `ValueError`   mas  não  saberemos  o motivo. Vamos acrescentar uma mensagem para deixar o erro mais claro:

        def deposita(self, valor):
            if(valor < 0):
                raise ValueError('Você tentou depositar um valor negativo')
            else:
                self._saldo += valor

3.  Faça o mesmo para o método  `saca()`  da classe  `ContaCorrente`, afinal o cliente também não pode sacar um valor negativo.

4.  Vamos validar também que o cliente não pode sacar um valor maior do que o saldo disponível em conta. Crie sua própria exceção chamada   `SaldoInsuficienteError`. Para isso, você precisa criar uma classe com esse nome que seja filha de `RuntimeError`.

        class SaldoInsuficienteError(RuntimeError):
            pass
No método `saca()` da classe  `ContaCorrente`  vamos utilizar esta nova exceção:
        class ContaCorrente(Conta):
            # código omitido
            def saca(self, valor):
                if(valor < 0):
                    raise ValueError('Você tentou sacar um valor negativo')
                if(self._saldo < valor):
                    raise SaldoInsuficienteError()
                    self._saldo -= (valor + 0.10)


### 11.8 OUTROS ERROS
Erros  de  sintaxe  Um  dos  erros  mais  comuns  é  o  SyntaxError.  Geralmente  suas  mensagens  não dizem muito, a mais comum é a   

    SyntaxError:  invalid  syntax

Por outro  lado, a  mensagem diz  o  local  onde  o  problema  ocorreu.  -  onde  o  Python  encontrou  o  problema.  São  descobertos quando  o  interpretador  está  traduzindo  o  código  fonte  para  o  bytecode.  Indicam  que  há  algo  de
errado com a estrutura do programa. Por exemplo: esquecer de fechar aspas, simples ou duplas, na hora  de  imprimir  um  mensagem;  esquecer  de  colocar  dois  pontos  (`:`)  ao  final  de  uma  instrução `if`, `while` ou `for` , etc...

Erro  semântico (?) Este  erro  é  quando  o  programa  não  se  comporta  como  esperado.  

Aqui  não  é lançada  uma  exceção,  o  programa  apenas  não  faz  a  coisa  certa.  São  mais  difíceis  de  encontrar porque o interpretador não fornece nenhuma informação já que não sabe o que o programa deveria fazer. São erros na regra de negócio. Utilizar a função  `print()`  em alguns lugares do código onde você suspeita que está gerando o erro pode ajudar.

### 11.9 PARA SABER MAIS - DEPURADOR DO PYTHON
O depurador do Python, o `pdb`, é um módulo embutido que funciona como um console interativo
onde é possível realizar debug de códigos python. Você pode ler mais a respeito na documentação:
https://docs.python.org/3/library/pdb.html

# CAPÍTULO 12
## COLLECTIONS
Objetivos:

* conhecer o módulo collections
* conhecer o módulo collections.abc

No  capítulo  4  vimos  uma  introdução  das  principais  estruturas  de  dados  do  Python  como  listas, tuplas, conjuntos e dicionários. Também aprendemos em orientação a objetos que tudo em Python é um objeto, inclusive essas estruturas.

O  Python  possui  uma  biblioteca  chamada    `collections` que  reúne  outros  tipos  de  dados alternativos ao já apresentados no capítulo 4. Esses tipos trazem novas funcionalidades.

O módulo `collections` também provê um módulo de classes abstratas, o módulo  `abc.collections`, que podem ser usadas para testar se determinada classe provê uma interface particular e aprenderemos um pouco sobre elas e seu uso.

### 12.1 USERLIST, USERDICT E USERSTRING
As estruturas de dados padrão do Python são de grande valia e muito utilizadas na linguagem, mas existem momentos que precisamos de funcionalidades extras que são comuns de projeto para projeto.

Nesse sentido surge o módulo  `collections`, pra acrescentar essas funcionalidades. Por exemplo, no Raspberry Pi ou Arduino, uma placa programada com pinos GPIO é representada
por  um  objeto  `board`  com  um  atributo  `pins`. Esse  atributo  contém  um  mapeamento  das  localizações físicas  dos  pinos  para  objetos  que  representam  os  pinos.  A  localização  física  pode  ser  um  número  ou
uma string como `A0` ou `B1`. Por consistência, é desejável que todas as chaves sejam strings assim como é conveniente que funcione para `pin[13]` quando o programador desejar fazer piscar o LED do pino 13.

Precisamos  usar  índices  que  são  strings,  portanto  um  dicionário.  Além  disso,  nosso  dicionário poderia apenas aceitar strings como chaves para este objetivo específico. Para não tratar isso durante a execução de nosso programa podemos criar uma classe que tenha o comportamento de um dicionário com essa característica específica.

Para  isso,  criamos  uma  classe  que  herda  de  uma  classe  chamada    `UserDict`    do  pacote `collections`:

In [0]:
class MeuDicionario(UserDict):
    pass

A  classe   `UserDict`   não  herda  de   `dict`   mas  simula  um  dicionário.  A   `UserDict`   possui  uma instância de  `dict`  interna chamada  `data`, que armazena os itens propriamente ditos.

Criar subclasses de tipos embutidos como   `dict`  ou   `list`  diretamente é propenso a erros porque seus  métodos  geralmente  ignoram  as  versões  sobrescritas.  Além  de  que  cada  implementação  pode  se comportar de maneira diferente. O fato de herdarmos de  `UserDict`  e não diretamente de  `dict`  é para
evitar esses problemas.

Criando a classe desta maneira, temos uma classe nossa que funciona como um dicionário. Mas não faz sentido criá-la sem acrescentar funcionalidades, já que o Python já possui essa estrutura pronta que é o  `dict`.

Vamos  criar  nosso  dicionário  de  modo  que  só  aceite  chaves  como   strings   e  vai  representar  os pinos da placa do Rasbperry Pi, por exemplo:

In [0]:
class Pins(UserDict):
    def __contains__(self, key):
        return str(key) in self.keys()
    def __setitem__(self, key, value):
        self.data[str(key)] = value

Note que a sobrescrita de   `__setitem__` garante que a chave sempre será uma `string`; Podemos testar essa classe:

In [0]:
pins = Pins(one=1)
print(pins)
pins[3] = 1
lista = [1, 2, 3]
pins(lista) = 2
print(pins)

Perceba que quando imprimimos o dicionário, todas suas chaves são  `strings`.

### 12.2 PARA SABER MAIS
Outros  tipos  que  existem  no  módulo  `collections`  são:    `defaultdict`, `counter`, `deque` e `namedtuple`.

Ao contrário do `dict`, no `defaultdict` não é necessário verificar se uma chave está presente ou não.

In [0]:
cores = [('1', 'azul'), ('2', 'amarelo'), ('3', 'vermelho'), ('1', 'branco'), ('3', 'verde')]
cores_favoritas = defaultdict(list)
for chave, valor in cores:
    cores_favoritas[chave].append(valor)
print(cores_favoritas)

Note que o código é executado sem acusar  `KeyError`.

#### Counter
O   `Counter`   é  um  contador  e  permite  contar  as  ocorrências  de  um  determinado  item  em  uma estrutura de dados:

In [0]:
from collections import Counter
cores = ['amarelo', 'azul', 'azul', 'vermelho', 'azul', 'verde', 'vermelho']
contador = Counter(cores)
print(contador)

Um   `Counter`  é um   `dict`  e pode receber um objeto iterável ou um mapa como argumento para realizar a contagem de seus elementos.


#### deque
O   `deque`  é uma estrutura de dados que fornece uma   fila   com  duas  extremidades  e  é  possível adicionar e remover elementos de ambos os lados:

In [0]:
from collections import deque
fila = deque()
fila.append('1')
fila.append('2')
fila.append('3')
print(len(fila))        #saída: 3
fila.pop()              #exclui elemento da direita
fila.append('3')        #adiciona elemento na direita
fila.popleft()          #exclui elemento da esquerda
fila.appendleft('1')    #adiciona elemento na esquerda

#### namedtuple
A   `namedtuple`, como o nome sugere, são tuplas nomeadas. Não é necessário usar índices inteiros para acessar seus elementos e podemos utilizar strings - similar aos dicionários. Mas ao contrários dos dicionários, `namedtuple`  é imutável:

In [0]:
from collections import namedtuple
Conta = namedtuple('Conta', 'numero titular saldo limite')
conta = Conta('123-4', 'João', 1000.0, 1000.0)
print(conta)
print(conta.titular)

Note  que  para  acessar  o  elemento  nomeado  utilizamos  o  operador  `.`  (ponto).  Uma   `namedtuple` posui dois argumentos obrigatórios que são: o nome da tupla e seus campos (separados por vírgula ou espaço).  No  exemplo,  a  tupla  se  chama   `Conta`   e  possui  4  campos:   `numero`, `titular`, `saldo` e `limite`. Como são imutáveis, não podemos modificar os valores de seus campos:

In [0]:
conta.titular = "José"

A  `namedtuple` também é compatível com uma tupla normal. Isso quer dizer que você também pode usar índices inteiros para acessar seus elementos.

In [0]:
print(conta[0])

Mais  detalhes  de  cada  uma  dessas estruturas  estão  na  documentação  e  pode  ser  acessada  por  este link: https://docs.python.org/3/library/collections.html.

Outra alternativa é usar a função `help(){ }` no
objeto para acessar a documentação.

### 12.3 COLLECTIONS ABC
O módulo   `collections.abc` fornece  classes  abstratas  que  podem  ser  usadas  para  testar  se  uma classe fornece uma interface específica. Por exemplo, se ela é iterável ou não. 

Imagine que o banco nos entregou um arquivo com vários funcionários e pediu que calculássemos a
bonificação  de  cada  um  deles.  Precisamos  acrescentar  este  arquivo  em  nossa  aplicação  para  iniciar  a leitura.

Conteúdo do arquivo funcionarios.txt:

    João,111111111-11,2500.0
    Jose,222222222-22,3500.0
    Maria,333333333-33,4000.0
    Pedro,444444444-44,2500.0
    Mauro,555555555-55,1700.0
    Denise,666666666-66,3000.0
    Tomas,777777777-77,4200.0

Cada linha do arquivo representa um   Funcionario  com seus atributos separados por vírgula. Este arquivo está no padrão Comma-separated-values, também conhecido como `csv` e são comumente usados.

O Python dá suporte de leitura para este tipo de arquivo. Então vamos acrescentar o módulo   `csv`  que vai ajudar na tarefa de ler o arquivo:

In [0]:
import csv
arquivo = open('funcionario.txt', 'r')
leitor = csv.reader(arquivo)
for linha in leitor:
    print(linha)
arquivo.open()

O  programa  acima  abre  um  arquivo  e  um  leitor  do  módulo `csv`, o `reader` -  recebe  o  arquivo como parâmetro e devolve um leitor que vai ler linha a linha e guardar seu conteúdo. Podemos iterar sobre este leitor e pedir para imprimir o conteúdo de cada linha - que é exatamente o que é feito no laço `for`. Por último fechamos o arquivo.

Repare que o `reader` guarda cada linha de um arquivo em uma lista e cada valor delimitado por vírgula se torna um elemento desta lista o que facilita o acesso aos dados.

Agora, com estes dados em mãos, podemos construir nossos objetos de tipo `Funcionario`:

In [0]:
for linha in reader:
    funcionario = Funcionario(linha[0], linha[1], linha[2])

Mas ainda precisamos de uma estrutura para guardá-los. Vamos utilizar uma lista:

In [0]:
funcionarios = []
for linha in reader:
    funcionario = Funcionario(linha[0], linha[1], linha[2])
    funcionarios.append(funcionario)

E por fim imprimimos os saldos da lista:

In [0]:
for f in funcionarios:
    print(f.saldo)

Acontece que nada impede, posteriormente, de inserirmos nesta lista qualquer outro objeto que não um funcionário:

In [0]:
funcionarios.append('Python')
funcionarios.append(1234)
funcionarios.append(True)

A `list` da  biblioteca  padrão  aceita  qualquer  tipo  de  objeto  como  elemento.

Não  queremos  este comportamento já que iremos calcular a bonificação de cada um deles e dependendo do tipo de objetos inserido na lista, gerará erros.

O ideal é que tivéssemos uma estrutura de dados que aceitasse apenas objetos de tipo `Funcionario`. O módulo  `collections.abc`  fornece classes abstratas que nos ajudam a construir estruturas específicas, com características da regra de negócio da aplicação.

### 12.4 CONSTRUINDO UM CONTAINER
O módulo  `collections.abc` possui uma classe absrata chamada  `Container` > Um   container  é
qualquer  objeto  que  contém  um  número arbitrário  de  outros  objetos.  Listas,  tuplas,  conjuntos  e dicionários  são  tipos  de  containers.  A  classe   `Container` suporta  o  operador `in` com  o  método `__contains__`.

Precisamos construir um container de objetos de tipo  `Funcionario`. Podemos construir uma classe que representará essa estrutura que deve ser subclasse de `Container`:

In [0]:
from collections.abc import Container
class Funcionarios(Container):
    pass

In [0]:
funcionarios = Funcionarios()

O código acima acusa um `TypeError`

Precisamos  implementar  o  método   `__contains__`   já  que   `Funcionarios`   deve  implementar  a classe abstrata   `Container`. A ideia é que nosso container se comporte como uma lista, então teremos
um atributo do tipo lista em nossa classe para guardar os objetos e implementar o método `contains`:

In [0]:
from collections.abc import Container
class Funcionarios(Container):
    _dados = []
    def __contains__(self, posicao):
        return self._dados.__contains__(self, posicao)

In [0]:
funcionarios = Funcionarios()

### 12.5 SIZED
O tamanho do nosso container também é uma informação importante. Nossa classe  `Funcionarios` deve saber retornar esse valor. 

Utilizamos a classe abstrata `Sized`  para garantir essa funcionalidade. A classe  `Sized`  provê o método  `len()`  através do método especial `__len__()`:

In [0]:
from collections.abc import Container
class Funcionarios(Container, Sized):
    _dados = []
    def __contains__(self, posicao):
        return self._dados.__contains__(self, posicao)
    def __len__(self):
        return len(self._dados)

In [0]:
funcionarios = Funcionarios()

### 12.6 ITERABLE
Além  de  conter  objetos  e  saber  retornar  a  quantidade  de  seus  elementos,  queremos  que  nosso container seja iterável, ou seja, que consigamos iterar sobre seus elementos em um laço for, por exemplo.

O módulo   `collection.abc`   também  provê  uma  classe  abstrata  para  este  comportamento,  é  a  classe `Iterable`. `Iterable`  suporta iteração com o método  `__iter__`:

In [0]:
from collections.abc import Container
class Funcionarios(Container, Sized, Iterable):
    _dados = []
    def __contains__(self, posicao):
        return self._dados.__contains__(self, posicao)
    def __len__(self):
        return len(self._dados)
    def __iter__(self):
        return self._dados.__iter__(self)

In [0]:
funcionarios = Funcionarios()

Toda coleção deve herdar dessas classes ABCs:  `Container`, `Iterable` e `Sized`. Ou implementar seus protocolos:  `__contains__`, `__iter__` e `__len__`.

Além  dessas  classes  existem  outras  que  facilitam  esse  trabalho  e  implementam  outros  protocolos.

Veja a hierarquia de classe do módulo `collections.abc`:

Figura 13.1: legenda da imagem


Além do que já foi implementado, a ideia é que nossa classe  `Funcionario` funcione como uma lista contando  apenas  objetos  do  tipo   `Funcionario`. Como  aprendemos  no  capítulo  4,  uma  lista  é  uma sequência. Além de uma sequência, é uma sequência mutável - podemos adicionar elementos em uma lista. Nossa classe  `Funcionario` também deve possuir essa funcionalidade.

Segundo o diagrama de classes do módulo  `collections.abc`, a classe que representa essa estrutura é  a    `MutableSequence`.  Note  que `MutableSequece` herda  de `Sequence` que representa  uma sequência; que por sua vez herda de `Container`, `Iterable` e `Sized`.

Figura 13.2: legenda da imagem

Portanto,  devemos  implementar  5  métodos  abstratos  (em  itálico  na  imagem)  segundo  a
documentação de   `MutableSequence`: `__len__`, `__getitem__`, `__setitem__`, `__delitem__` e `insert`. O método  `__getitem__`  garante que a classe é um  `Container`  e  `Iterable`. 

Segundo a `PEP 234` (https://www.python.org/dev/peps/pep-0234/)  um  objeto  pode  ser  iterável  com  um  laço  `for`  se implementa  `__iter__` ou `__getitem__`.

Então, basta nossa classe `Funcionario` herdar de `MutableSequence` e implementar seus métodos
abstratos:

In [0]:
class Funcionarios(MutableSequence):
    _dados = []
    def __len__(self):
        return len(self._dados)
    def __getitem__(self, posicao):
        return self._dados[posicao]
    def __setitem__(self, posicao, valor):
        self._dados[posicao] = valor
    def __delitem__(self, posicao):
        del self._dados[posicao]
    def insert(self, posicao, valor):
        return self._dados.insert(posicao, valor)

E  podemos  voltar  ao  nosso  código  para  acrescentar  os  dados  de  um  arquivo  em  nosso  container `Funcionarios`:

In [0]:
import csv
arquivo = open('funcionario.txt', 'r')
leitor = csv.reader(arquivo)
funcionarios = Funcionarios()
for linha in leitor:
    funcionario = Funcionario(linha[0], linha[1], linha[2])
    funcionarios.append(funcionario)
arquivo.open()

O método   `insert()` garante  o funcionamento  do  método `append()`. E  podemos  imprimir  os valores dos salários de cada funcionário:

In [0]:
for f in funcionarios:
    print(f.salario)

Mas até aqui não há nada de diferente de uma lista comum. Ainda não há nada que impeça de inserir qualquer outro objeto em nossa lista. Nossa classe `Funcionarios`  se comporta como uma lista comum.

A  ideia  de  implementarmos  as  interfaces  de   `collections.abc`   era  exatamente  modificar  alguns comportamentos.

Queremos que nossa lista de funcionários apenas aceite objetos  `Funcionario`. Vamos sobrescrever os métodos  `__setitem__()`  que atribuiu um valor em determinada posição na lista. Este método pode apenas atribuir a uma determinada posição um objeto  `Funcionario`.

Para isso, vamos usar o método `isinstance()` que vai verificar se o objeto a ser atribuído é uma instância  de `Funcionario`.  Caso  contrário,  vamos  lançar  uma  exceção    `TypeError` com  uma mensagem de erro:

In [0]:
def __setitem__(self, posicao, valor):
    if (isinstance(valor, Funcionario)):
        self._dados[posicao] = valor
    else:
        raise ValueError('Valor atribuído não é um Funcionario')

Agora,  ao  tentar  atribuir  uma  valor  a  determinada  posição  de  nossa  lista,  recebemos  um `TypeError`:

In [0]:
funcionarios[0] = 'Python'

Faremos o mesmo com o método  `insert()`:

In [0]:
def insert(self, posicao, valor):
    if(isinstance(valor, Funcionario)):
        return self._dados.insert(posicao, valor)
    else:
        raise ValueError('Valor inserido não é um Funcionario')

E podemos testar nossa classe imprimindo não apenas o salário mas o valor da bonificação de cada `Funcionario` através do método `get_bonificacao()`  que definimos nos capítulos passados:

In [0]:
import csv
arquivo = open('funcionario.txt', 'r')
leitor = csv.reader(arquivo)
funcionarios = Funcionarios()
for linha in leitor:
    funcionario = Funcionario(linha[0], linha[1], linha[2])
    funcionarios.append(funcionario)
print('salário - bonificação')    
for c in contas:
    print('{} - {}'.formar(f.salario, f.get_bonificacao()))    
arquivo.open()

As classes ABCs foram criadas para encapsular conceitos genéricos e abstrações como aprendemos
no  capítulo  de  classes  abstratas.  São  comumente  utilizadas  em  grandes  aplicações  e  frameworks  para garantir a consistência do sistema através dos métodos  `isinstance()` e `issubclass()`. No dia a dia é raramente usado e basta o uso correto das estruturas já fornecidas pela biblioteca padrão do Python para
a maior parte das tarefas.

Conhecer o módulo collections.abc é (hahahah estava assim na apostila!)

### 12.7 EXERCÍCIO: CRIANDO NOSSA SEQUÊNCIA
1.  Vá na pasta no curso e copie o arquivo  contas.txt  na pasta   src  do projeto   banco  que contém vários dados de contas correntes de clientes do banco.

2.  Crie um arquivo chamado contas.py  na  pasta   src   do  projeto   banco .  Crie  uma  classe  chamada Contas  que herde da classe abstrata  MutableSequence :

        from collections.abc import Sequence
        class Contas(MutableSequence):
            pass
3.  Vamos criar um atributo da classe do tipo  list  para armazenar nossas contas:
 from collections.abc import MutableSequence
 class Contas(MutableSequence):
     _dados = []
4.  Tente instanciar um objeto de tipo  Contas :
.
 if __name__=='__main__':
     contas = Contas()
Note  que  não  podemos  instanciar  este  objeto.  A  interface    MutableSequence    nos  obriga  a
implementar alguns métodos:
 Traceback (most recent call last):
   File <stdin>, line 44, in <module>
     contas = Contas()
 TypeError: Can't instantiate abstract class Contas with abstract methods __delitem__, __getitem__
, __len__, __setitem__, insert
5.  Implemente os métodos exigidos pela interface  MutableSequence  na classe  Contas :
 from collections.abc import MutableSequence
 class Contas(MutableSequence):
     _dados = []
     def __len__(self):
         return len(self._dados)
     def __getitem__(self, posicao):
         return self._dados[posicao]
     def __setitem__(self, posicao, valor):
         self._dados[posicao] = valor
     def __delitem__(self, posicao):
         del self._dados[posicao]
     def insert(self, posicao, valor):
         return self._dados.insert(posicao, valor)
Agora conseguimos instanciar nossa classe sem nenhum erro:
 if __name__=='__main__':
     contas = Contas()
6.  Nossa sequência só deve permitir adicionar elementos que sejam do tipo  Conta . Vamos acrescentar
essa validação nos métodos   __setitem__  e   insert . Caso o valor não seja uma   Conta ,  vamos
lançar um  ValueError  com as devidas mensagens de erro:
 def __setitem__(self, posicao, valor):
     if (isinstance(valor, Conta)):
         self._dados[posicao] = valor
     else:
         raise ValueError("valor atribuído não é uma conta")
 def insert(self, posicao, valor):
     if(isinstance(valor, Conta)):
         return self._dados.insert(posicao, valor)
     else:
         raise ValueError('valor inserido não é uma conta')
7.  Vamos iniciar a leitura dos dados do arquivo para armazenar em nosso objeto  contas :
 if __name__=='__main__':
     import csv
     contas = Contas()
     arquivo = open('contas.txt', 'r')
     leitor = csv.reader(arquivo)
     arquivo.close()
8.  Vamos  criar  uma  laço  for  para  ler  cada  linha  do  arquivo  e  construir  um  objeto  do  tipo
 ContaCorrente .
 if __name__=='__main__':
     import csv
     from conta import ContaConrrete
     contas = Contas()
     arquivo = open('contas.txt', 'r')
     leitor = csv.reader(arquivo)
     for linha in leitor:
         conta = ContaCorrente(linha[0], linha[1], linha[2], linha[3])
     arquivo.close()
9.  Queremos inserir cada conta criada em nossa sequência mutável   contas . Vamos pedir para que o
programa acrescente cada conta criada em contas:
 for linha in leitor:
     conta = ContaCorrente(linha[0], linha[1], float(linha[2]))
     contas.append(conta)
 arquivo.close()
10.  Nossa  classe   Contas   implementa   MutableSequence .  Isso  quer  dizer  que  ela  é  iterável  já  que
 MutableSequence  implementa o protocolo   __iter__  através do método   __getitem__ . Vamos
iterar  através  de  uma  laço  for  nosso  objeto   contas   e  pedir  para  imprimir  o  saldo  e  o  valor  do
imposto de cada uma delas:
 if __name__ == '__main__':
     #código omitido
     arquivo.close()
     print('saldo -  imposto')
     for c in contas:
         print('{} - {}'.format(c.saldo, c.get_valor_imposto()))
Que vai gerar a saída:
 saldo  - imposto
 1200.0 - 12.0
 2200.0 - 22.0
 1500.0 - 15.0
 5300.0 - 53.0
 7800.0 - 78.0
.
 1700.0 - 17.0
 2300.0 - 23.0
 8000.0 - 80.0
 4600.0 - 46.0
 9400.0 - 94.0
11.  (Opcional)  Modifique  o  código  do  exercício  anterior  de  modo  que  imprima  o  valor  do  saldo
atualizado das contas.
12.  (Opcional)  Faça  o  mesmo  com  as  contas  poupanças.  Crie  um  arquivo  com  extensão   .csv   com
algumas  contas  poupanças,  faça  a  leitura,  construa  os  objetos  e  acrescente  em  uma  estrutura  de
dados do tipo  MutableSequence .
13.  (Opcional) Refaça o exercício utilizando  MutableMapping  ao invés de  MutableSequence .

# CAPÍTULO 13
## APÊNDICE - PYTHON2 OU PYTHON3?
Caso  você  esteja  iniciando  seus  estudos na  linguagem  Python  ou  começando  um  projeto  novo, aconselhamos fortemente que você utilize o Python3.

O Python2 vem sendo chamado de Python legado ou Python antigo por boa parte da comunidade que  está  em  constante  atividade  para  fazer  a  migração  da  base  de  código  existente (bem  grande,  por sinal) para Python3.

Aconselhamos a leitura deste artigo para maiores detalhes:https://wiki.python.org/moin/Python2orPython3.

A pergunta correta aqui é: Quando devo usar o Python antigo? E a resposta mais comum que você
vai encontrar é: use Python antigo quando você não tiver escolha.

Por exemplo, quando você trabalhar em um projeto antigo e migrar para a nova versão não for uma alternativa  no  momento.  Ou  quando  você  precisa  utilizar  uma  biblioteca  que  ainda  não  funciona  no Python3  ou  não  está  em  processo  de  mudança.  Outro  caso  é  quando  seu  servidor  de  hospedagem  só
permite usar Python2 - aqui o aconselhável é procurar por outro serviço que atenda sua demanda.

No mais, você encontrará muito material sobre o Python2 na internet e aos poucos vai conhecendo
melhor as diferenças entre uma versão e outra.

### 13.1 QUAIS AS DIFERENÇAS?
Neste artigo você vai encontrar a resposta https://docs.python.org/3/whatsnew/3.0.html. Mas neste capítulo mostramos as diferenças mais básicas e importantes para você iniciar seus estudos.

### 13.2 A FUNÇÃO PRINT()
No  Python2  o  comando   `print`   funciona  de  maneira  diferente  já  que  não  é  uma  função.  Para imprimir algo fazemos:
    # no python2
    print "Hello World!"

No Python3  `print`  é uma função e utilizamos os parênteses como delimitadores:

In [0]:
print("Hello World!")

### 13.3 A FUNÇÃO INPUT()
A função  `raw_input`  do Python2 foi renomeada para  `input()`  no Python3:
    # no python2
    nome = raw_input("Digite seu nome: ")

No Python3:

In [0]:
nome = input("Digite seu nome: ")

### 13.4 DIVISÃO DECIMAL
No Python2 a divisão entre números decimais é diferente entre um número decimal e um inteiro:
    # no python2
    >>> 5 / 2
    2
    >>> 5 / 2.0
    2.5

No  Python3  a  divisão  tem  o  mesmo  comportamento  da  matemática.  E  se  quisermos  o  resultado
inteiro da divisão utilizamos  `//`:

In [0]:
5 / 2

In [0]:
5 // 2

### 13.5 HERANÇA
No Python2 suas classes devem herdar de  `object`:
    # no python2
    class MinhaClasse(object):
        def metodo(self, attr1, attr2):
            return attr1 + attr2

No Python3 essa herança é implícita, não precisando herdar explicitamente de `object`:

In [0]:
class MinhaClasse():
    def metodo(self, attr1, attr2):
        return attr1 + attr2

# CAPÍTULO 14
## APÊNDICE - INSTALAÇÃO
O Python já vem instalado nos sistemas Linux e Mac OS mas será necessário fazer o download da última versão (Python 3.6) para acompanhar a apostila. O Python não vem instalado por padrão no Windows e o download deverá ser feito no site https://www.python.org/ além de algumas configurações extras.

### 14 .1 INSTALANDO O PYTHON NO WINDOWS
O primeiro passo é acessar o site do Python: https://www.python.org/. Na sessão de   Downloads  já será  disponibilizado  o  instalador  específico  do  Windows automaticamente,  portanto  é  só  baixar  o Python3, na sua versão mais atual.

Figura 15.1: Tela de download do Python para o Windows

Após  o  download  ser  finalizado,  abra-o  e  na  primeira  tela  marque  a  opção   

    Add Python  3.X  to PATH. 
    
Essa opção é importante para conseguirmos executar o Python dentro do Prompt de Comando
do Windows. Caso você não tenha marcado esta opção, terá que configurar a variável de ambiente no Windows de forma manual.

Figura 15.2: Checkbox selecionado

Selecione a instalação customizada somente para ver a instalação com mais detalhes.

Figura 15.3: Instalação customizada

Na  tela  seguinte  são  as  features opcionais,  se  certifique  que  o  gerenciador  de  pacotes   pip   esteja selecionado,  ele  que  permite  instalar pacotes  e  bibliotecas  no  Python.  Clique em   Next   para  dar seguimento na instalação.

Figura 15.4: Optional Features

Já  na  terceira  tela,  deixe  tudo  como  está,  mas  se  atente  ao  diretório  de  instalação  do  Python, para caso queira procurar o executável ou algo que envolva o seu diretório.

Figura 15.5: Advanced Options

Por fim, basta clicar em  `Install`  e aguardar o término da instalação.

Figura 15.6: Instalação Concluída com Sucesso

Terminada a instalação, teste se o Python foi instalado corretamente. Abra o Prompt de Comando e execute:
    
    python -V

OBS: Para que funcione corretamente é necessário que seja no Prompt de Comando e não em algum programa Git Bash instalado em sua máquina. E o comando  `python -V  é importante que esteja com o `V` com letra maiúscula.

Esse comando imprime a versão do Python instalada no Windows. Se a versão for impressa, significa que o Python foi instalado corretamente. Agora, rode o comando  `python`:
    
    python

Assim você terá acesso ao console do próprio Python, conseguindo assim utilizá-lo.

### 14.2 INSTALANDO O PYTHON NO LINUX
Os sistemas operacionais baseados no Debian já possuem o Python3 pré-instalado. Verifique se o seu sistema já possui o Python3 instalado executando o seguinte comando no terminal:
    
    python3 -V

OBS: O comando  `python3 -V`  é importante que esteja com o  `V`  com letra maiúscula.

Este comando retorna a versão do Python3 instalada. Se você ainda não tiver ele instalado, digite os seguintes comandos no terminal:

    sudo apt-get update
    sudo apt3-get install python

Para que você consiga instalar os pacotes do Python é necessário ter o gerenciador de pacotes   `pip` instalado no sistema. Para instalar esse gerenciador, digite no terminal:
    
    sudo apt-get install python-pip

### 14.3 INSTALANDO O PYTHON NO MACOS
A maneira mais fácil de instalar o Python3 no MacOS é utilizando o  `Homebrew`. Com o   `Homebrew` instalado, abra o terminal e digite os seguintes comandos:

    brew update
    brew install python3

Para que você consiga instalar os pacotes do Python é necessário ter o gerenciador de pacotes   `pip` instalado no sistema. Para instalar esse gerenciador, digite no terminal:

    sudo apt-get install python-pip

### 14.4 OUTRAS FORMAS DE UTILIZAR O PYTHON
Podemos rodar o Python diretamente do seu próprio Prompt.

Podemos  procurar  pelo  Python  na  caixa  de  pesquisa  do  Windows  e  abri-lo,  assim  o seu  console próprio será aberto. Uma outra forma é abrir a `IDLE` do Python, que se parece muito com o console mas vem com um menu que possui algumas opções extras.